# 🇺🇸🇧🇷 USD/BRL Options — Análise Interativa
Notebook para gerar gráficos de **Delta**, **Gamma Exposure**, **Curvatura do Gamma**, **Open Interest por Strike** e **Midwalls**.

In [1]:
try:
    import plotly
except ImportError:
    import sys, subprocess
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'plotly'])
import pandas as pd
import numpy as np
import re, datetime as dt, os
from pathlib import Path
from scipy.stats import norm
from scipy.interpolate import PchipInterpolator
from scipy.optimize import brentq
import plotly.graph_objects as go


In [2]:
CONTRACT_MULT = 50000
SPOT = 5.46750
IV_ANNUAL = 0.1383
RISK_FREE = 0.05
DATAREF = dt.date.today()
USE_IMPLIED_VOL = False
USE_CSV_SPOT = False
HVL_ANNUAL = 0.1383
SIGMA_FACTOR = 1.0
USE_HVL_FLIP = True
USE_CSV_SPOT = False
ATM_BAND_STEPS = 0.5
DPI_WEIGHTS = {'delta':0.25,'gamma':0.25,'charm':0.25,'vanna':0.25}
DPI_WINDOW_STRIKES = 2
EXTERNAL_PUT_OI_TOTAL = None
EXTERNAL_CALL_OI_TOTAL = None
EXTERNAL_PUT_PREM_TOTAL = None
EXTERNAL_CALL_PREM_TOTAL = None


In [3]:
def _num(s):
    return pd.to_numeric(
        s.astype(str).str.replace(r'[^\d\.\-]', '', regex=True).str.replace(',', ''),
        errors='coerce'
    )

def collect_csv_files():
    roots = [Path.cwd(), Path('/content'), Path('/mnt/data')]
    files = [str(p) for r in roots if r.exists() for p in r.glob('*.csv')]
    if not files:
        try:
            from google.colab import files as colab_files  # type: ignore
            up = colab_files.upload()
            if up: files = list(up.keys())
        except Exception:
            pass
    if not files:
        raise FileNotFoundError('⚠️ Nenhum .csv encontrado.')
    return files

def read_options_table(path: Path):
    try:
        df = pd.read_csv(path)
    except Exception:
        return None, None
    if df.empty:
        return None, None
    df.columns = [c.strip() for c in df.columns]
    spot_val = None
    for c in df.columns:
        if any(k in c.lower() for k in ['spot', 'underlying', 'à vista', 'avista']):
            try:
                spot_val = float(_num(df[c]).dropna().iloc[0])
                break
            except Exception:
                pass
    rename_map = {
        'Open Int':'Open Int', 'Open Interest':'Open Int', 'OI':'Open Int',
        'Qtde. Contratos em Aberto':'Open Int',
        'Option Type':'OptionType', 'Type':'OptionType'
    }
    df = df.rename(columns={c: rename_map[c] for c in df.columns if c in rename_map})
    if not {'Strike','Open Int','OptionType'}.issubset(df.columns):
        return None, spot_val
    df['OptionType'] = (df['OptionType'].astype(str).str.strip().str.upper()
                        .replace({'C':'CALL','CALLS':'CALL','P':'PUT','PUTS':'PUT'}))
    for col in ['Strike','Last','Volume','Open Int','Premium']:
        if col in df.columns:
            df[col] = _num(df[col])
    df = df.dropna(subset=['Strike','Open Int']).reset_index(drop=True)
    df['StrikeK'] = df['Strike'].astype(float)
    return df, spot_val


In [4]:
CSV_FILES = collect_csv_files()
parts, spot_auto = [], None
for f in CSV_FILES:
    d, s = read_options_table(Path(f))
    if d is not None and not d.empty:
        parts.append(d)
    if s and not spot_auto:
        spot_auto = s
if not parts:
    raise FileNotFoundError('⚠️ Nenhuma planilha válida encontrada.')
options = pd.concat(parts, ignore_index=True)
if USE_CSV_SPOT and spot_auto: SPOT = spot_auto
if SPOT < 100: SPOT *= 1000
print('Spot (pontos):', SPOT)


Spot (pontos): 5467.5


In [5]:
expiry = None
m = re.search(r'exp[_\-]?(20\d{2})[_\-](\d{2})[_\-](\d{2})', os.path.basename(CSV_FILES[0]))
if m:
    yyyy, mm, dd = map(int, m.groups())
    expiry = dt.date(yyyy, mm, dd)
bdays = int(np.busday_count(DATAREF, expiry)) if expiry else 1
T = max(bdays, 1) / 252
strikes_ref = np.sort(options['StrikeK'].unique())
min_k, max_k = strikes_ref.min(), strikes_ref.max()
pad = max(10, int((max_k - min_k) * 0.03))
print('Nº strikes:', len(strikes_ref), '| Range:', (min_k, max_k))


Nº strikes: 6 | Range: (np.float64(5225.0), np.float64(5650.0))


In [6]:
oi_call = options.loc[options['OptionType']=='CALL'].groupby('StrikeK', as_index=True)['Open Int'].sum()
oi_put  = options.loc[options['OptionType']=='PUT'].groupby('StrikeK', as_index=True)['Open Int'].sum()
oi_call_ref = np.array([oi_call.get(k, 0.0) for k in strikes_ref], dtype=float)
oi_put_ref  = np.array([oi_put.get(k, 0.0)  for k in strikes_ref], dtype=float)
midwalls_strikes = (strikes_ref[:-1] + strikes_ref[1:]) / 2
midwalls_call = (oi_call_ref[:-1] + oi_call_ref[1:]) / 2
midwalls_put  = (oi_put_ref[:-1]  + oi_put_ref[1:]) / 2
print('Midwalls criadas:', len(midwalls_strikes))
def bs_price(S,K,T,r,sigma,typ):
    if S<=0 or K<=0 or T<=0 or sigma<=0: return np.nan
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    if typ=='C':
        return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    else:
        return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
def implied_vol(price,S,K,T,r,typ):
    if np.isnan(price) or price<=0: return np.nan
    lo, hi = 1e-6, 3.0
    for _ in range(40):
        mid = 0.5*(lo+hi)
        pm = bs_price(S,K,T,r,mid,typ)
        if np.isnan(pm): return np.nan
        if pm>price: hi = mid
        else: lo = mid
    return 0.5*(lo+hi)
def _group_price(df, typ, col1='Last', col2='Premium'):
    sel = df.loc[df['OptionType']==typ]
    series = None
    if col1 in sel.columns and not sel[col1].isna().all():
        series = sel.groupby('StrikeK')[col1].mean()
    elif col2 in sel.columns and not sel[col2].isna().all():
        series = sel.groupby('StrikeK')[col2].mean()
    else:
        series = pd.Series(dtype=float)
    return np.array([series.get(k, np.nan) for k in strikes_ref], dtype=float)
prices_call = _group_price(options,'CALL')
prices_put  = _group_price(options,'PUT')
iv_call_ref = np.array([implied_vol(prices_call[i], SPOT, float(k), T, RISK_FREE, 'C') if USE_IMPLIED_VOL else IV_ANNUAL for i,k in enumerate(strikes_ref)])
iv_put_ref  = np.array([implied_vol(prices_put[i],  SPOT, float(k), T, RISK_FREE, 'P') if USE_IMPLIED_VOL else IV_ANNUAL for i,k in enumerate(strikes_ref)])
iv_pair = np.vstack([iv_call_ref, iv_put_ref])
iv_strike_ref = np.nanmean(iv_pair, axis=0)
iv_strike_ref = np.where(np.isnan(iv_strike_ref), IV_ANNUAL, iv_strike_ref)
iv_strike_ref = np.where(iv_strike_ref>2.0, iv_strike_ref/100.0, iv_strike_ref)

def get_iv_daily_atm():
    idx_atm = int(np.argmin(np.abs(strikes_ref - SPOT)))
    iva = float(iv_strike_ref[idx_atm]) if 'iv_strike_ref' in globals() else IV_ANNUAL
    if np.isnan(iva) or iva<=0 or iva>1.5:
        iva = IV_ANNUAL
    base = (iva if USE_IMPLIED_VOL else IV_ANNUAL)
    return base/np.sqrt(252)
step = float(np.median(np.diff(strikes_ref))) if len(strikes_ref)>1 else 1.0
atm_band = step*ATM_BAND_STEPS
def _class_call(k):
    if k < SPOT - atm_band: return 'ITM'
    if abs(k - SPOT) <= atm_band: return 'ATM'
    return 'OTM'
def _class_put(k):
    if k > SPOT + atm_band: return 'ITM'
    if abs(k - SPOT) <= atm_band: return 'ATM'
    return 'OTM'
labels_call = np.array([_class_call(float(k)) for k in strikes_ref])
labels_put  = np.array([_class_put(float(k))  for k in strikes_ref])
def _sum_by(labels, arr, key):
    m = labels==key
    return float(np.nansum(arr[m]))
pcr_itm = (_sum_by(labels_put, oi_put_ref, 'ITM') / max(_sum_by(labels_call, oi_call_ref, 'ITM'), 1e-9))
pcr_atm = (_sum_by(labels_put, oi_put_ref, 'ATM') / max(_sum_by(labels_call, oi_call_ref, 'ATM'), 1e-9))
pcr_otm = (_sum_by(labels_put, oi_put_ref, 'OTM') / max(_sum_by(labels_call, oi_call_ref, 'OTM'), 1e-9))
def vega(S,K,T,r,sigma):
    if S<=0 or K<=0 or T<=0 or sigma<=0: return np.nan
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    return S*norm.pdf(d1)*np.sqrt(T)
vega_ref = np.array([0 if np.isnan(vega(SPOT, float(k), T, RISK_FREE, float(s))) else vega(SPOT, float(k), T, RISK_FREE, float(s)) for k,s in zip(strikes_ref, iv_strike_ref)])
vex_tot = vega_ref*(oi_call_ref + oi_put_ref)
iv_skew = np.zeros_like(iv_strike_ref)
if len(strikes_ref) >= 3:
    for i in range(1, len(strikes_ref)-1):
        iv_skew[i] = (iv_strike_ref[i+1] - iv_strike_ref[i-1])/(strikes_ref[i+1]-strikes_ref[i-1])
    iv_skew[0] = (iv_strike_ref[1] - iv_strike_ref[0])/(strikes_ref[1]-strikes_ref[0])
    iv_skew[-1] = (iv_strike_ref[-1] - iv_strike_ref[-2])/(strikes_ref[-1]-strikes_ref[-2])


Midwalls criadas: 5


In [7]:
def greeks(S, K, T, r, sigma, t):
    if S<=0 or K<=0 or T<=0 or sigma<=0:
        return np.nan, np.nan
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    delta = norm.cdf(d1) if t=='C' else norm.cdf(d1) - 1
    gamma = norm.pdf(d1) / (S*sigma*np.sqrt(T))
    return delta, gamma

dC,gC,dP,gP = [],[],[],[]
for K in strikes_ref:
    d,g = greeks(SPOT, K, T, RISK_FREE, IV_ANNUAL,'C'); dC.append(0 if np.isnan(d) else d); gC.append(0 if np.isnan(g) else g)
    d,g = greeks(SPOT, K, T, RISK_FREE, IV_ANNUAL,'P'); dP.append(0 if np.isnan(d) else d); gP.append(0 if np.isnan(g) else g)
dC,gC,dP,gP = map(np.array, (dC,gC,dP,gP))
dT = 1/252
dsigma = 0.01
chC,chP,vaC,vaP = [],[],[],[]
for K in strikes_ref:
    dTp,_ = greeks(SPOT, K, max(T + dT, 1e-6), RISK_FREE, IV_ANNUAL, 'C')
    dTm,_ = greeks(SPOT, K, max(T - dT, 1e-6), RISK_FREE, IV_ANNUAL, 'C')
    chC.append(((0 if np.isnan(dTp) else dTp) - (0 if np.isnan(dTm) else dTm)) / (2*dT))
    dTp,_ = greeks(SPOT, K, max(T + dT, 1e-6), RISK_FREE, IV_ANNUAL, 'P')
    dTm,_ = greeks(SPOT, K, max(T - dT, 1e-6), RISK_FREE, IV_ANNUAL, 'P')
    chP.append(((0 if np.isnan(dTp) else dTp) - (0 if np.isnan(dTm) else dTm)) / (2*dT))
    dSp,_ = greeks(SPOT, K, T, RISK_FREE, IV_ANNUAL + dsigma, 'C')
    dSm,_ = greeks(SPOT, K, T, RISK_FREE, max(IV_ANNUAL - dsigma, 1e-6), 'C')
    vaC.append(((0 if np.isnan(dSp) else dSp) - (0 if np.isnan(dSm) else dSm)) / (2*dsigma))
    dSp,_ = greeks(SPOT, K, T, RISK_FREE, IV_ANNUAL + dsigma, 'P')
    dSm,_ = greeks(SPOT, K, T, RISK_FREE, max(IV_ANNUAL - dsigma, 1e-6), 'P')
    vaP.append(((0 if np.isnan(dSp) else dSp) - (0 if np.isnan(dSm) else dSm)) / (2*dsigma))
chC,chP,vaC,vaP = map(np.array, (chC,chP,vaC,vaP))
gex_tot  = gC*oi_call_ref*CONTRACT_MULT*SPOT*0.01 + gP*oi_put_ref*CONTRACT_MULT*SPOT*0.01
gex_cum  = np.cumsum(gex_tot)
# Base assinada para cálculo de flips: sinal por moneyness
sgn_call = np.where(strikes_ref <= SPOT, +1.0, -1.0)
sgn_put  = np.where(strikes_ref >= SPOT, -1.0, +1.0)
gex_flip_base = (gC*oi_call_ref*sgn_call + gP*oi_put_ref*sgn_put) * CONTRACT_MULT * SPOT * 0.01
gex_cum_signed = np.cumsum(gex_flip_base)
# PVOP alternativo (valor por ponto/prêmio) — se não definido, usa o peso atual como padrão
PVOP = float(PVOP) if 'PVOP' in globals() else float(CONTRACT_MULT)*float(SPOT)*0.01
gex_flip_base_pvop = (gC*oi_call_ref*sgn_call + gP*oi_put_ref*sgn_put) * PVOP
gex_cum_signed_pvop = np.cumsum(gex_flip_base_pvop)
ivw = iv_strike_ref if 'iv_strike_ref' in globals() else np.ones_like(strikes_ref, dtype=float)
gex_oi = gC*oi_call_ref + gP*oi_put_ref
gex_vol = gex_oi * ivw
gex_oi_cum = np.cumsum(gex_oi)
gex_vol_cum = np.cumsum(gex_vol)
r_gamma = gex_flip_base_pvop
dexp_tot = dC*oi_call_ref + dP*oi_put_ref
dexp_cum = np.cumsum(dexp_tot)
charm_tot = chC*oi_call_ref + chP*oi_put_ref
vanna_tot = vaC*oi_call_ref + vaP*oi_put_ref
charm_cum = np.cumsum(charm_tot)
vanna_cum = np.cumsum(vanna_tot)
def _norm(a):
    m = float(np.nanmax(np.abs(a))) if np.nanmax(np.abs(a))>0 else 1.0
    return a/m
dpi_arr = (DPI_WEIGHTS['delta']*_norm(dexp_tot) + DPI_WEIGHTS['gamma']*_norm(gex_tot) + DPI_WEIGHTS['charm']*_norm(charm_tot) + DPI_WEIGHTS['vanna']*_norm(vanna_tot))
i_spot = int(np.argmin(np.abs(strikes_ref - SPOT)))
i0 = max(0, i_spot - DPI_WINDOW_STRIKES)
i1 = min(len(strikes_ref)-1, i_spot + DPI_WINDOW_STRIKES)
dealer_pressure_spot = float(np.nanmean(dpi_arr[i0:i1+1]))
try:
    gamma_flip = strikes_ref[np.where(gex_cum_signed>=0)[0][0]]
except Exception:
    gamma_flip = None
regime = 'Gamma Positivo' if (gamma_flip is not None and SPOT>=gamma_flip) else 'Gamma Negativo'
regime_color = 'lime' if 'Positivo' in regime else 'red'
delta_agregado_val = float(np.nansum(dexp_tot))
iv_atm = float(iv_strike_ref[int(np.argmin(np.abs(strikes_ref - SPOT)))]) if 'iv_strike_ref' in globals() else IV_ANNUAL
iv_atm = (IV_ANNUAL if np.isnan(iv_atm) else iv_atm)
IV_DAILY = get_iv_daily_atm()
range_low, range_high = SPOT*(1 - IV_DAILY), SPOT*(1 + IV_DAILY)
dist_call = float(np.min(np.abs(strikes_ref - SPOT)))
if 'CALL' in options['OptionType'].unique():
    idx_call = np.argsort(oi_call_ref)[-1:]
    if len(idx_call)>0: dist_call = float(np.min(np.abs(strikes_ref[idx_call] - SPOT)))
dist_put = float(np.min(np.abs(strikes_ref - SPOT)))
if 'PUT' in options['OptionType'].unique():
    idx_put = np.argsort(oi_put_ref)[-1:]
    if len(idx_put)>0: dist_put = float(np.min(np.abs(strikes_ref[idx_put] - SPOT)))
dist_wall_min = max(min(dist_call, dist_put), 1e-9)
range_wall_ratio = float((range_high - range_low)/dist_wall_min)
# Calcula Gamma Flip (HVL) neste bloco para evitar NameError na Info
gamma_flip_hvl = None
if USE_HVL_FLIP:
    try:
        HVL_DAILY = float(HVL_ANNUAL)/np.sqrt(252)
        step = float(np.median(np.diff(strikes_ref))) if len(strikes_ref)>1 else 25.0
        sigma_pts = float(SIGMA_FACTOR) * max(step*2.0, float(SPOT)*float(HVL_DAILY))
        order = np.argsort(np.array(strikes_ref, dtype=float))
        ks = np.array(strikes_ref, dtype=float)[order]
        gex = np.array(gex_tot, dtype=float)[order]
        w = np.exp(-((ks - float(SPOT))**2) / (2.0 * (sigma_pts**2)))
        gex_cum_hvl = np.cumsum(gex * w)
        sg_h = np.sign(gex_cum_hvl)
        idx_h = np.where(np.diff(sg_h)!=0)[0]
        if len(idx_h)>0:
            j = int(np.argmin(np.abs(ks[idx_h] - float(SPOT))))
            i = idx_h[j]
            y1, y2 = gex_cum_hvl[i], gex_cum_hvl[i+1]
            x1, x2 = ks[i], ks[i+1]
            gamma_flip_hvl = float(x1 if y2==y1 else x1 - y1*(x2 - x1)/(y2 - y1))
        else:
            gamma_flip_hvl = float(ks[int(np.argmin(np.abs(gex_cum_hvl)))])
    except Exception:
        gamma_flip_hvl = None
# Gamma Flip (HVL janela local) calculado antes de construir labels para evitar NameError
gamma_flip_hvl_win = None
if USE_HVL_FLIP:
    try:
        HVL_DAILY = float(HVL_ANNUAL)/np.sqrt(252)
        step = float(np.median(np.diff(strikes_ref))) if len(strikes_ref)>1 else 25.0
        W = float(SIGMA_FACTOR) * max(step*2.0, float(SPOT)*float(HVL_DAILY))
        order = np.argsort(np.array(strikes_ref, dtype=float))
        ks_all = np.array(strikes_ref, dtype=float)[order]
        gex_all = np.array(gex_tot, dtype=float)[order]
        mask = (ks_all >= float(SPOT) - W) & (ks_all <= float(SPOT) + W)
        ks = ks_all[mask]
        gex = gex_all[mask]
        if len(ks) >= 2:
            gex_cum_w = np.cumsum(gex)
            sgw = np.sign(gex_cum_w)
            idxw = np.where(np.diff(sgw)!=0)[0]
            if len(idxw)>0:
                j = int(np.argmin(np.abs(ks[idxw] - float(SPOT))))
                i = idxw[j]
                y1, y2 = gex_cum_w[i], gex_cum_w[i+1]
                x1, x2 = ks[i], ks[i+1]
                gamma_flip_hvl_win = float(x1 if y2==y1 else x1 - y1*(x2 - x1)/(y2 - y1))
            else:
                gamma_flip_hvl_win = float(ks[int(np.argmin(np.abs(gex_cum_w)))])
        else:
            gamma_flip_hvl_win = None
    except Exception:
        gamma_flip_hvl_win = None
info = (f"<b>USD/BRL Spot:</b> <span style='color:lime'>{SPOT:.0f}</span><br>"
        f"<b>Delta Agregado:</b> {delta_agregado_val:,.0f}<br>"
        f"<b>Gamma Flip:</b> {(f'{gamma_flip:.0f}' if gamma_flip is not None else 'N/A')}<br>"
        
        f"<b>Regime:</b> <span style='color:{regime_color}'>{regime}</span><br>"
        f"<b>Vol Diária:</b> <span style='color:yellow'>{IV_DAILY*100:.2f}%</span><br>"
        f"<b>Dealer Pressure:</b> {dealer_pressure_spot:+.2f}<br>"
        f"<b>Range/Distância Walls:</b> {range_wall_ratio:.2f}")


In [8]:
fig = go.Figure()
fig.add_trace(go.Bar(x=strikes_ref, y=dexp_tot, name='Delta Agregado', marker_color=['lime' if v>=0 else 'red' for v in dexp_tot], visible=True))
fig.add_trace(go.Scatter(x=strikes_ref, y=dexp_cum, mode='lines', name='Delta Acumulado', visible=False, line=dict(width=3, color='yellow')))
fig.add_trace(go.Bar(x=strikes_ref, y=gex_tot, name='Gamma Exposure', marker_color='cyan', opacity=0.6, visible=False))
fig.add_trace(go.Scatter(x=strikes_ref, y=gex_cum, mode='lines', name='Curvatura do Gamma (acumulado)', line=dict(color='orange', width=3), visible=False))
fig.add_trace(go.Bar(x=strikes_ref, y=oi_call_ref, name='CALL OI', marker_color='blue', visible=False))
fig.add_trace(go.Bar(x=strikes_ref, y=-oi_put_ref, name='PUT OI', marker_color='red', visible=False))
fig.add_trace(go.Bar(x=midwalls_strikes, y=midwalls_call, marker_color='#2c2c2c', opacity=0.8, visible=False, showlegend=False))
fig.add_trace(go.Bar(x=midwalls_strikes, y=-midwalls_put, marker_color='#2c2c2c', opacity=0.8, visible=False, showlegend=False))
fig.add_trace(go.Bar(x=strikes_ref, y=charm_tot, name='Charm Exposure', marker_color='magenta', opacity=0.6, visible=False))
fig.add_trace(go.Bar(x=strikes_ref, y=vanna_tot, name='Vanna Exposure', marker_color='purple', opacity=0.6, visible=False))
fig.add_trace(go.Scatter(x=strikes_ref, y=charm_cum, mode='lines', name='Charm Acumulado', line=dict(color='magenta', width=3), visible=False))
fig.add_trace(go.Scatter(x=strikes_ref, y=vanna_cum, mode='lines', name='Vanna Acumulado', line=dict(color='purple', width=3), visible=False))
fig.add_trace(go.Bar(x=strikes_ref, y=vex_tot, name='Vega Exposure', marker_color='gold', opacity=0.6, visible=False))
fig.add_trace(go.Scatter(x=strikes_ref, y=iv_skew, mode='lines', name='Skew IV (local)', line=dict(color='white', width=2, dash='dot'), visible=False))
fig.add_trace(go.Scatter(x=strikes_ref, y=dpi_arr, mode='lines', name='Dealer Pressure', line=dict(color='#9ca3af', width=3), visible=False))
fig.update_layout(template='plotly_dark', barmode='overlay', xaxis_title='Strike', yaxis_title='Exposição / OI', xaxis=dict(range=[SPOT-300, SPOT+300], tickmode='auto'), legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='left', x=0.0), margin=dict(t=100))
spot_line = dict(type='line', x0=SPOT, x1=SPOT, y0=0, y1=1, xref='x', yref='paper', line=dict(color='lime', dash='dot', width=2))
hline0   = dict(type='line', x0=min_k, x1=max_k, y0=0, y1=0, line=dict(color='white', dash='dot', width=1))
flip_line = None
flip_label = None
spot_label = dict(x=float(SPOT), y=0.92, xref='x', yref='paper', text=f'SPOT {SPOT:.0f}', showarrow=False, font=dict(color='lime', size=12), bgcolor='black', bordercolor='lime')
if gamma_flip is not None:
    flip_line = dict(type='line', x0=gamma_flip, x1=gamma_flip, y0=0, y1=1, xref='x', yref='paper', line=dict(color='red', dash='dash', width=2))
    flip_label = dict(x=gamma_flip, y=0.05, xref='x', yref='paper', text='Gamma Flip', showarrow=False, font=dict(color='red', size=12), bgcolor='black', bordercolor='red')
def infobox():
    return dict(xref='paper', yref='paper', x=0.99, y=0.95, xanchor='right', showarrow=False, align='left', text=info, font=dict(size=14, color='white'), bordercolor=regime_color, borderwidth=2, borderpad=6, bgcolor='rgba(20,20,20,0.85)', opacity=0.95)
buttons = [
    dict(label='Delta Agregado', method='update', args=[{'visible':[True, False, False, False, False, False, False, False, False, False, False, False, False, False, False]}, {'title':'USD/BRL — Delta Agregado por Strike', 'shapes':[spot_line, hline0], 'annotations':[infobox(), spot_label]}]),
    dict(label='Delta Acumulado', method='update', args=[{'visible':[False, True, False, False, False, False, False, False, False, False, False, False, False, False, False]}, {'title':'USD/BRL — Delta Acumulado por Strike', 'shapes':[spot_line, hline0], 'annotations':[infobox(), spot_label]}]),
    dict(label='Gamma Exposure', method='update', args=[{'visible':[False, False, True, True, False, False, False, False, False, False, False, False, False, False, False]}, {'title':'USD/BRL — Gamma Exposure (total + curvatura)', 'shapes':[spot_line, hline0] + ([flip_line] if flip_line else []), 'annotations':([infobox(), spot_label, flip_label] if flip_label else [infobox(), spot_label])}]),
    dict(label='EDI — Open Interest por Strike', method='update', args=[{'visible':[False, False, False, False, True, True, True, True, False, False, False, False, False, False, False]}, {'title':'EDI — Open Interest (OI) por Strike', 'shapes':[spot_line, hline0] + ([flip_line] if flip_line else []), 'annotations':([infobox(), spot_label, flip_label] if flip_label else [infobox(), spot_label])}]),
    dict(label='Charm Exposure', method='update', args=[{'visible':[False, False, False, False, False, False, False, False, True, False, False, False, False, False, False]}, {'title':'USD/BRL — Charm Exposure por Strike', 'shapes':[spot_line, hline0], 'annotations':[infobox(), spot_label]}]),
    dict(label='Vanna Exposure', method='update', args=[{'visible':[False, False, False, False, False, False, False, False, False, True, False, False, False, False, False]}, {'title':'USD/BRL — Vanna Exposure por Strike', 'shapes':[spot_line, hline0], 'annotations':[infobox(), spot_label]}]),
    dict(label='Charm Acumulado', method='update', args=[{'visible':[False, False, False, False, False, False, False, False, False, False, True, False, False, False, False]}, {'title':'USD/BRL — Charm Acumulado', 'shapes':[spot_line, hline0], 'annotations':[infobox(), spot_label]}]),
    dict(label='Vanna Acumulado', method='update', args=[{'visible':[False, False, False, False, False, False, False, False, False, False, False, True, False, False, False]}, {'title':'USD/BRL — Vanna Acumulado', 'shapes':[spot_line, hline0], 'annotations':[infobox(), spot_label]}]),
    dict(label='Vega Exposure', method='update', args=[{'visible':[False, False, False, False, False, False, False, False, False, False, False, False, True, False, False]}, {'title':'USD/BRL — Vega Exposure por Strike', 'shapes':[spot_line, hline0], 'annotations':[infobox(), spot_label]}]),
    dict(label='Skew IV (local)', method='update', args=[{'visible':[False, False, False, False, False, False, False, False, False, False, False, False, False, True, False]}, {'title':'USD/BRL — Skew IV (local)', 'shapes':[spot_line, hline0], 'annotations':[infobox(), spot_label]}]),
    dict(label='Dealer Pressure', method='update', args=[{'visible':[False, False, False, False, False, False, False, False, False, False, False, False, False, False, True]}, {'title':'USD/BRL — Dealer Pressure (normalizado)', 'shapes':[spot_line, hline0], 'annotations':[infobox(), spot_label]}]),
    dict(label='Visão Completa', method='relayout', args=[{'xaxis.range':[min_k - pad, max_k + pad]}])
]
fig.update_layout(updatemenus=[dict(direction='down', showactive=True, x=0.0, y=1.05, buttons=buttons)])
fig.show()


In [9]:
top_n = 3
idx_call = np.argsort(oi_call_ref)[-top_n:]
idx_put  = np.argsort(oi_put_ref)[-top_n:]
call_walls = strikes_ref[idx_call]
put_walls  = strikes_ref[idx_put]
call_walls_oi = oi_call_ref[idx_call]
put_walls_oi  = oi_put_ref[idx_put]

iv_atm = float(iv_strike_ref[int(np.argmin(np.abs(strikes_ref - SPOT)))])
iv_atm = (IV_ANNUAL if np.isnan(iv_atm) else iv_atm)
IV_DAILY = get_iv_daily_atm()
range_low, range_high = SPOT*(1 - IV_DAILY), SPOT*(1 + IV_DAILY)
range_lines = [
    dict(type='line', x0=float(range_low),  x1=float(range_low),  y0=0, y1=1, xref='x', yref='paper',
         line=dict(color='yellow', dash='dot', width=1)),
    dict(type='line', x0=float(range_high), x1=float(range_high), y0=0, y1=1, xref='x', yref='paper',
         line=dict(color='yellow', dash='dot', width=1)),
]

sg = np.sign(gex_cum)
idx = np.where(np.diff(sg)!=0)[0]
zero_gamma = None
if len(idx)>0:
    i = idx[0]
    y1, y2 = gex_cum[i], gex_cum[i+1]
    x1, x2 = strikes_ref[i], strikes_ref[i+1]
    zero_gamma = x1 - y1*(x2 - x1)/(y2 - y1) if y2!=y1 else x1

gamma_flip = zero_gamma if zero_gamma is not None else (strikes_ref[np.where(gex_cum_signed>=0)[0][0]] if np.any(gex_cum_signed>=0) else None)

call_top = sorted(list(zip(call_walls, call_walls_oi)), key=lambda x: x[1], reverse=True)
put_top  = sorted(list(zip(put_walls,  put_walls_oi)),  key=lambda x: x[1], reverse=True)
walls_call_txt = ' | '.join([f"{k:.0f}({v:,.0f})" for k,v in call_top])
walls_put_txt  = ' | '.join([f"{k:.0f}({v:,.0f})" for k,v in put_top])

regime = 'Gamma Positivo' if (gamma_flip is not None and SPOT>=gamma_flip) else 'Gamma Negativo'
regime_color = 'lime' if 'Positivo' in regime else 'red'
delta_agregado_val = float(np.nansum(dexp_tot))

info = (f"<b>DOLFUT:</b> <span style='color:lime'>{SPOT:.0f}</span><br>"
        f"<b>Delta Agregado:</b> {delta_agregado_val:,.0f}<br>"
        f"<b>Gamma Flip:</b> {(f'{gamma_flip:.0f}' if gamma_flip is not None else 'N/A')}<br>"
        f"<b>Regime:</b> <span style='color:{regime_color}'>{regime}</span><br>"
        f"<b>Vol Diária:</b> <span style='color:yellow'>{IV_DAILY*100:.2f}%</span><br>"
        f"<b>CALL walls:</b> {walls_call_txt}<br>"
        f"<b>PUT walls:</b> {walls_put_txt}")

fig2 = go.Figure()
fig2.add_trace(go.Bar(x=strikes_ref, y=dexp_tot, name='Delta Agregado',
                      marker_color=['lime' if v>=0 else 'red' for v in dexp_tot], visible=True))
fig2.add_trace(go.Scatter(x=strikes_ref, y=dexp_cum, mode='lines', name='Delta Acumulado',
                          visible=False, line=dict(width=3, color='yellow')))
fig2.add_trace(go.Bar(x=strikes_ref, y=gex_tot, name='Gamma Exposure', marker_color='cyan',
                      opacity=0.6, visible=False))
fig2.add_trace(go.Scatter(x=strikes_ref, y=gex_cum, mode='lines', name='Curvatura do Gamma (acumulado)',
                          line=dict(color='orange', width=3), visible=False))
fig2.add_trace(go.Bar(x=strikes_ref, y=oi_call_ref, name='CALL OI', marker_color='blue', visible=False))
fig2.add_trace(go.Bar(x=strikes_ref, y=-oi_put_ref, name='PUT OI', marker_color='red', visible=False))
fig2.add_trace(go.Bar(x=midwalls_strikes, y=midwalls_call, marker_color='#2c2c2c', opacity=0.8,
                      visible=False, showlegend=False))
fig2.add_trace(go.Bar(x=midwalls_strikes, y=-midwalls_put, marker_color='#2c2c2c', opacity=0.8,
                      visible=False, showlegend=False))

fig2.update_layout(template='plotly_dark', barmode='overlay',
                   xaxis_title='Strike', yaxis_title='Contratos Abertos',
                   xaxis=dict(range=[SPOT-300, SPOT+300], tickmode='auto'),
                   legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='left', x=0.0),
                   margin=dict(t=100))

spot_line = dict(type='line', x0=float(SPOT), x1=float(SPOT), y0=0, y1=1, xref='x', yref='paper',
                 line=dict(color='lime', dash='dot', width=2))
hline0   = dict(type='line', x0=float(min_k), x1=float(max_k), y0=0, y1=0,
                line=dict(color='white', dash='dot', width=1))

flip_line = None
flip_label = None
spot_label = dict(x=float(SPOT), y=0.92, xref='x', yref='paper', text=f'SPOT {SPOT:.0f}', showarrow=False, font=dict(color='lime', size=12), bgcolor='black', bordercolor='lime')
if gamma_flip is not None:
    flip_line = dict(type='line', x0=float(gamma_flip), x1=float(gamma_flip), y0=0, y1=1,
                     xref='x', yref='paper', line=dict(color='red', dash='dash', width=2))
    flip_label = dict(x=float(gamma_flip), y=0.05, xref='x', yref='paper', text='Gamma Flip',
                      showarrow=False, font=dict(color='red', size=12),
                      bgcolor='black', bordercolor='red')

wall_lines = ([dict(type='line', x0=float(k), x1=float(k), y0=0, y1=1, xref='x', yref='paper',
                    line=dict(color='blue', dash='dot', width=1)) for k in call_walls]
              + [dict(type='line', x0=float(k), x1=float(k), y0=0, y1=1, xref='x', yref='paper',
                      line=dict(color='red', dash='dot', width=1)) for k in put_walls])

def infobox2():
    return dict(xref='paper', yref='paper', x=0.99, y=0.95, xanchor='right', showarrow=False,
                align='left', text=info, font=dict(size=14, color='white'),
                bordercolor=regime_color, borderwidth=2, borderpad=6,
                bgcolor='rgba(20,20,20,0.85)', opacity=0.95)

buttons2 = [
    dict(label='Delta Agregado', method='update',
         args=[{'visible':[True, False, False, False, False, False, False, False]},
               {'title':'USD/BRL — Delta Agregado (nova célula)',
                'shapes':[spot_line, hline0] + range_lines,
                'annotations':[infobox2(), spot_label]}]),
    dict(label='Delta Acumulado', method='update',
         args=[{'visible':[False, True, False, False, False, False, False, False]},
               {'title':'USD/BRL — Delta Acumulado (nova célula)',
                'shapes':[spot_line, hline0] + range_lines,
                'annotations':[infobox2(), spot_label]}]),
    dict(label='Gamma Exposure', method='update',
         args=[{'visible':[False, False, True, True, False, False, False, False]},
               {'title':'Edi - Dólar "Análise de Open Interest"',
                'shapes':[spot_line, hline0] + range_lines + ([flip_line] if flip_line else []),
                'annotations':([infobox2(), spot_label, flip_label] if flip_label else [infobox2(), spot_label])}]),
    dict(label='Open Interest por Strike', method='update',
         args=[{'visible':[False, False, False, False, True, True, True, True]},
               {'title':'USD/BRL — OI por Strike (nova célula)',
                'shapes':[spot_line, hline0] + range_lines + (wall_lines if wall_lines else []),
                'annotations':[infobox2(), spot_label]}]),
    dict(label='Visão Completa', method='relayout',
         args=[{'xaxis.range':[float(min_k) - pad, float(max_k) + pad]}]),
]

overlayButtons = [
    dict(label='Overlay: Range + Walls + ZeroGamma', method='relayout',
         args=[{'shapes':[spot_line, hline0] + range_lines
                          + (wall_lines if wall_lines else [])
                          + ([flip_line] if flip_line else [])}]),
    dict(label='Overlay: Range apenas', method='relayout',
         args=[{'shapes':[spot_line, hline0] + range_lines}]),
    dict(label='Overlay: Walls apenas', method='relayout',
         args=[{'shapes':[spot_line, hline0] + (wall_lines if wall_lines else [])}]),
    dict(label='Overlay: GammaFlip apenas', method='relayout',
         args=[{'shapes':[spot_line, hline0] + ([flip_line] if flip_line else [])}]),
    dict(label='Limpar overlays', method='relayout',
         args=[{'shapes':[spot_line, hline0]}]),
]
buttonsUnified = buttons2 + overlayButtons

BRAND_TITLE = 'EDI — Painel Delta & GEX'
fig2.update_layout(updatemenus=[
    dict(type='dropdown', direction='down', showactive=True, active=0, x=1.00, y=1.30, buttons=buttonsUnified, bgcolor='rgba(30,30,30,0.95)', bordercolor='#444', borderwidth=1, font=dict(color='#e5e7eb', size=12), pad=dict(t=4, r=4, b=4, l=4))
], title=dict(text=BRAND_TITLE, font=dict(color='white', size=18), x=0.5))
fig2.show()
items = ['Spot','Range baixo','Range alto','Gamma Flip','Regime','CALL walls top','PUT walls top']
values = [f'{SPOT:.0f}', f'{range_low:.0f}', f'{range_high:.0f}', (f'{gamma_flip:.0f}' if gamma_flip is not None else 'N/A'), regime, walls_call_txt, walls_put_txt]
descs  = ['Preço à vista (pontos)', 'Limite inferior esperado intradiário', 'Limite superior esperado intradiário', 'Zero Gamma (Gamma Flip) interpolado', 'Sinal do Gamma acumulado no spot', 'Top-3 paredes de OI em CALL', 'Top-3 paredes de OI em PUT']
fig_vals = go.Figure(data=[go.Table(
    header=dict(values=['Item','Valor','Descrição'], fill_color='grey', align='left', font=dict(color='white', size=12)),
    cells=dict(values=[items, values, descs], fill_color='black', align='left', font=dict(color='white', size=12))
)])
fig_vals.update_layout(template='plotly_dark', margin=dict(t=30))
fig_vals.show()

In [10]:
top_n = 3
idx_call = np.argsort(oi_call_ref)[-top_n:]
idx_put  = np.argsort(oi_put_ref)[-top_n:]
call_walls = strikes_ref[idx_call]
put_walls  = strikes_ref[idx_put]
call_walls_oi = oi_call_ref[idx_call]
put_walls_oi  = oi_put_ref[idx_put]

iv_atm = float(iv_strike_ref[int(np.argmin(np.abs(strikes_ref - SPOT)))])
iv_atm = (IV_ANNUAL if np.isnan(iv_atm) else iv_atm)
IV_DAILY = get_iv_daily_atm()
range_low, range_high = SPOT*(1 - IV_DAILY), SPOT*(1 + IV_DAILY)
range_lines = [
    dict(type='line', x0=float(range_low),  x1=float(range_low),  y0=0, y1=1, xref='x', yref='paper',
         line=dict(color='yellow', dash='dot', width=1)),
    dict(type='line', x0=float(range_high), x1=float(range_high), y0=0, y1=1, xref='x', yref='paper',
         line=dict(color='yellow', dash='dot', width=1)),
]


sg = np.sign(gex_cum_signed)
idx = np.where(np.diff(sg)!=0)[0]
zero_gamma = None
if len(idx)>0:
    zg_list = []
    for i in idx:
        y1, y2 = gex_cum_signed[i], gex_cum_signed[i+1]
        x1, x2 = strikes_ref[i], strikes_ref[i+1]
        xz = x1 - y1*(x2 - x1)/(y2 - y1) if y2!=y1 else x1
        zg_list.append(xz)
    zero_gamma = float(zg_list[int(np.argmin(np.abs(np.array(zg_list) - SPOT)))])

gamma_flip = zero_gamma if zero_gamma is not None else (float(strikes_ref[np.where(gex_cum_signed>=0)[0][0]]) if np.any(gex_cum_signed>=0) else None)
# HVL Gamma Flip: pondera exposições de gamma por distância ao spot usando HVL diária
gamma_flip_hvl = None
if USE_HVL_FLIP:
    try:
        HVL_DAILY = float(HVL_ANNUAL)/np.sqrt(252)
        sigma_pts = max(float(np.median(np.diff(strikes_ref))) * 2.0 if len(strikes_ref)>1 else 25.0, float(SPOT)*HVL_DAILY)
        order = np.argsort(np.array(strikes_ref, dtype=float))
        ks = np.array(strikes_ref, dtype=float)[order]
        gex = np.array(gex_tot, dtype=float)[order]
        w = np.exp(-((ks - float(SPOT))**2) / (2.0 * (sigma_pts**2)))
        gex_cum_hvl = np.cumsum(gex * w)
        sg_h = np.sign(gex_cum_hvl)
        idx_h = np.where(np.diff(sg_h)!=0)[0]
        if len(idx_h)>0:
            # escolhe o cruzamento mais próximo do spot e interpola
            j = int(np.argmin(np.abs(ks[idx_h] - float(SPOT))))
            i = idx_h[j]
            y1, y2 = gex_cum_hvl[i], gex_cum_hvl[i+1]
            x1, x2 = ks[i], ks[i+1]
            gamma_flip_hvl = float(x1 if y2==y1 else x1 - y1*(x2 - x1)/(y2 - y1))
        else:
            gamma_flip_hvl = float(ks[int(np.argmin(np.abs(gex_cum_hvl)))])
    except Exception:
        gamma_flip_hvl = None

# Gamma Flip (HVL log-moneyness) calculado antes de labels para evitar NameError
gamma_flip_hvl_log = None
if USE_HVL_FLIP:
    try:
        HVL_DAILY = float(HVL_ANNUAL)/np.sqrt(252)
        order = np.argsort(np.array(strikes_ref, dtype=float))
        ks = np.array(strikes_ref, dtype=float)[order]
        gex = np.array(gex_tot, dtype=float)[order]
        sigma_m = float(HVL_DAILY) * float(SIGMA_FACTOR)
        z = np.log(ks/float(SPOT))
        w = np.exp(-(z**2) / (2.0 * (sigma_m**2)))
        gex_cum_log = np.cumsum(gex * w)
        sg_l = np.sign(gex_cum_log)
        idx_l = np.where(np.diff(sg_l)!=0)[0]
        if len(idx_l)>0:
            j = int(np.argmin(np.abs(ks[idx_l] - float(SPOT))))
            i = idx_l[j]
            y1, y2 = gex_cum_log[i], gex_cum_log[i+1]
            x1, x2 = ks[i], ks[i+1]
            gamma_flip_hvl_log = float(x1 if y2==y1 else x1 - y1*(x2 - x1)/(y2 - y1))
        else:
            gamma_flip_hvl_log = float(ks[int(np.argmin(np.abs(gex_cum_log)))])
    except Exception:
        gamma_flip_hvl_log = None
call_top = sorted(list(zip(call_walls, call_walls_oi)), key=lambda x: x[1], reverse=True)
put_top  = sorted(list(zip(put_walls,  put_walls_oi)),  key=lambda x: x[1], reverse=True)
walls_call_txt = ' | '.join([f"{float(k):.0f}({float(v):,.0f})" for k,v in call_top])
walls_put_txt  = ' | '.join([f"{float(k):.0f}({float(v):,.0f})" for k,v in put_top])

call_near_idx = idx_call[int(np.argmin(np.abs(call_walls - SPOT)))]
put_near_idx  = idx_put[int(np.argmin(np.abs(put_walls  - SPOT)))]
nearest_call_txt = f"{float(strikes_ref[call_near_idx]):.0f} (dist {abs(float(strikes_ref[call_near_idx])-float(SPOT)):.0f})"
nearest_put_txt  = f"{float(strikes_ref[put_near_idx]):.0f} (dist {abs(float(strikes_ref[put_near_idx])-float(SPOT)):.0f})"

regime = 'Gamma Positivo' if (gamma_flip is not None and SPOT>=gamma_flip) else 'Gamma Negativo'
regime_color = 'lime' if 'Positivo' in regime else 'red'
delta_agregado_val = float(np.nansum(dexp_tot))
pcr = (float(np.nansum(oi_put_ref)) / float(np.nansum(oi_call_ref))) if float(np.nansum(oi_call_ref))>0 else np.nan

info = (f"<b>DOLFUT:</b> <span style='color:lime'>{SPOT:.0f}</span><br>"
        f"<b>Delta Agregado:</b> {delta_agregado_val:,.0f}<br>"
        f"<b>Gamma Flip:</b> {(f'{gamma_flip:.0f}' if gamma_flip is not None else 'N/A')}<br>"
        f"<b>Regime:</b> <span style='color:{regime_color}'>{regime}</span><br>"
        f"<b>Volatilidade Diária:</b> <span style='color:yellow'>{IV_DAILY*100:.2f}%</span><br>"
        f"<b>Put/Call Ratio:</b> {(f'{pcr:.2f}' if not np.isnan(pcr) else 'N/A')}<br>"
        f"<b>CALL walls:</b> {walls_call_txt}<br>"
        f"<b>PUT walls:</b> {walls_put_txt}<br>"
        f"<b>CALL wall próxima:</b> {nearest_call_txt}<br>"
        f"<b>PUT wall próxima:</b> {nearest_put_txt}")

fig3 = go.Figure()
fig3.add_trace(go.Bar(x=strikes_ref, y=dexp_tot, name='Delta Agregado',
                      marker_color=['lime' if v>=0 else 'red' for v in dexp_tot], visible=True,
                      text=[f'{float(k):.0f}' for k in strikes_ref], textposition='outside',
                      textfont=dict(size=11, color='white'), cliponaxis=False,
                      hovertemplate='Strike %{x:.0f}<br>Valor %{y:.0f}'))
fig3.add_trace(go.Scatter(x=strikes_ref, y=dexp_cum, mode='lines', name='Delta Acumulado',
                          visible=False, line=dict(width=3, color='yellow')))
fig3.add_trace(go.Bar(x=strikes_ref, y=gex_tot, name='Gamma Exposure', marker_color='cyan',
                      opacity=0.6, visible=False, hovertemplate='Strike %{x:.0f}<br>Valor %{y:.0f}'))
fig3.add_trace(go.Scatter(x=strikes_ref, y=gex_cum, mode='lines', name='Curvatura do Gamma (acumulado)',
                          line=dict(color='orange', width=3), visible=False, hovertemplate='Strike %{x:.0f}<br>Valor %{y:.0f}'))
fig3.add_trace(go.Bar(x=strikes_ref, y=oi_call_ref, name='CALL OI', marker_color='lime', visible=False, text=[f'{float(k):.0f}' for k in strikes_ref], textposition='outside', textfont=dict(size=10, color='white'), cliponaxis=False, hovertemplate='Strike %{x:.0f}<br>OI %{y:.0f}'))
fig3.add_trace(go.Bar(x=strikes_ref, y=-oi_put_ref, name='PUT OI', marker_color='red', visible=False, text=[f'{float(k):.0f}' for k in strikes_ref], textposition='outside', textfont=dict(size=10, color='white'), cliponaxis=False, hovertemplate='Strike %{x:.0f}<br>OI %{y:.0f}'))
fig3.add_trace(go.Bar(x=midwalls_strikes, y=midwalls_call, marker_color='lime', opacity=0.2, visible=False, showlegend=False))
fig3.add_trace(go.Bar(x=midwalls_strikes, y=-midwalls_put, marker_color='red', opacity=0.2, visible=False, showlegend=False))
fig3.add_trace(go.Bar(x=strikes_ref, y=charm_tot, name='Charm Exposure', marker_color='magenta', opacity=0.6, visible=False, hovertemplate='Strike %{x:.0f}<br>Valor %{y:.0f}'))
fig3.add_trace(go.Bar(x=strikes_ref, y=vanna_tot, name='Vanna Exposure', marker_color='purple', opacity=0.6, visible=False, hovertemplate='Strike %{x:.0f}<br>Valor %{y:.0f}'))
fig3.add_trace(go.Bar(x=strikes_ref, y=gex_oi, name='Gamma Exposure (OI)', marker_color='cyan', opacity=0.6, visible=False, hovertemplate='Strike %{x:.0f}<br>Valor %{y:.0f}'))
fig3.add_trace(go.Bar(x=strikes_ref, y=gex_vol, name='Gamma Exposure (IV)', marker_color='violet', opacity=0.6, visible=False, hovertemplate='Strike %{x:.0f}<br>Valor %{y:.0f}'))
fig3.add_trace(go.Bar(x=strikes_ref, y=r_gamma, name='R Gamma (PVOP assinado)', marker_color='teal', opacity=0.6, visible=False, hovertemplate='Strike %{x:.0f}<br>Valor %{y:.0f}'))
fig3.add_trace(go.Scatter(x=strikes_ref, y=gex_oi_cum, mode='lines', name='Curvatura do GEX OI (acumulado)', line=dict(color='yellow', width=2), visible=False, hovertemplate='Strike %{x:.0f}<br>Valor %{y:.0f}'))
fig3.add_trace(go.Scatter(x=strikes_ref, y=gex_vol_cum, mode='lines', name='Curvatura do GEX VOL (acumulado)', line=dict(color='violet', width=2), visible=False, hovertemplate='Strike %{x:.0f}<br>Valor %{y:.0f}'))

fig3.update_layout(template='plotly_dark', barmode='overlay',
                   xaxis_title='Strike', yaxis_title='Contratos Abertos',
                   xaxis=dict(range=[SPOT-300, SPOT+300], tickmode='auto'),
                   legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='left', x=0.0),
                   margin=dict(t=100))

spot_line = dict(type='line', x0=float(SPOT), x1=float(SPOT), y0=0, y1=1, xref='x', yref='paper',
                 line=dict(color='lime', dash='dot', width=2))
hline0   = dict(type='line', x0=float(min_k), x1=float(max_k), y0=0, y1=0,
                line=dict(color='white', dash='dot', width=1))

flip_line = None
flip_label = None
if gamma_flip is not None:
    flip_line = dict(type='line', x0=float(gamma_flip), x1=float(gamma_flip), y0=0, y1=1,
                     xref='x', yref='paper', line=dict(color='red', dash='dash', width=2))
    flip_label = dict(x=float(gamma_flip), y=0.05, xref='x', yref='paper', text='Gamma Flip',
                      showarrow=False, font=dict(color='red', size=12), bgcolor='black', bordercolor='red')

    flip_label_hvl_win = dict(x=float(gamma_flip_hvl_win), y=0.14, xref='x', yref='paper', text='Gamma Flip (HVL janela)',
                               showarrow=False, font=dict(color='magenta', size=12), bgcolor='black', bordercolor='magenta')

wall_lines = ([dict(type='line', x0=float(k), x1=float(k), y0=0, y1=1, xref='x', yref='paper',
                    line=dict(color='blue', dash='dot', width=1)) for k in call_walls]
              + [dict(type='line', x0=float(k), x1=float(k), y0=0, y1=1, xref='x', yref='paper',
                      line=dict(color='red', dash='dot', width=1)) for k in put_walls])

def infobox3():
    return dict(xref='paper', yref='paper', x=0.995, y=0.98, xanchor='right', showarrow=False,
                align='left', text=info, font=dict(size=15, color='white'),
                bordercolor='#2563eb', borderwidth=2, borderpad=10,
                bgcolor='rgba(20,20,20,0.88)', opacity=0.98)

range_low_label = dict(x=float(range_low), y=0.98, xref='x', yref='paper', text='Mínima Diária',
                       showarrow=False, font=dict(color='yellow', size=11), bgcolor='black')
range_high_label = dict(x=float(range_high), y=0.98, xref='x', yref='paper', text='Máxima Diária',
                        showarrow=False, font=dict(color='yellow', size=11), bgcolor='black')
mid_above_idx = np.where(midwalls_strikes >= SPOT)[0]
mid_below_idx = np.where(midwalls_strikes <  SPOT)[0]
mid_above = list(map(lambda x: f'{float(x):.0f}', list(midwalls_strikes[mid_above_idx][:3])))
mid_below = list(map(lambda x: f'{float(x):.0f}', list(midwalls_strikes[mid_below_idx][-3:])))
mid_above_txt = ' | '.join(mid_above) if len(mid_above)>0 else 'N/A'
mid_below_txt = ' | '.join(mid_below) if len(mid_below)>0 else 'N/A'

call_fence_lines = [dict(type='line', x0=float(k), x1=float(k), y0=0, y1=1, xref='x', yref='paper', line=dict(color='lime', dash='dot', width=1)) for k in call_walls]
put_fence_lines  = [dict(type='line', x0=float(k), x1=float(k), y0=0, y1=1, xref='x', yref='paper', line=dict(color='red',  dash='dot', width=1)) for k in put_walls]
midwall_lines_shapes = [dict(type='line', x0=float(k), x1=float(k), y0=0, y1=1, xref='x', yref='paper', line=dict(color='#6b7280', dash='dot', width=1)) for k in midwalls_strikes]
midwall_annos = [dict(x=float(k), y=0.92 - 0.04*(i%4), xref='x', yref='paper', text=f'{float(k):.0f}', showarrow=False, font=dict(color='#9ca3af', size=11), bgcolor='black') for i,k in enumerate(midwalls_strikes)]
fib_percs = [0.236, 0.382, 0.618, 0.7645]
fib_lines_shapes = []
fib_annos = []
for i in range(len(strikes_ref)-1):
    lower = float(strikes_ref[i]); upper = float(strikes_ref[i+1]); dist = upper - lower
    for j, p in enumerate(fib_percs):
        lvl = lower + p*dist
        fib_lines_shapes.append(dict(type='line', x0=lvl, x1=lvl, y0=0, y1=1, xref='x', yref='paper', line=dict(color='#374151', dash='dot', width=1)))
        fib_annos.append(dict(x=lvl, y=0.90 - 0.05*(j%4), xref='x', yref='paper', text=f'{lvl:.0f}', showarrow=False, font=dict(color='#d1d5db', size=10), bgcolor='black'))
shapes_strike_mid_fibo = [spot_line, hline0] + call_fence_lines + put_fence_lines + midwall_lines_shapes + fib_lines_shapes
annos_strike_mid_fibo  = [infobox3(), spot_label] + midwall_annos + fib_annos
buttonsMainNew = [
    dict(label='Delta Agregado', method='update',
         args=[{'visible':[True, False, False, False, False, False, False, False, False, False, False, False, False]},
               {'title':{'text':'EDI — Delta Agregado (Soma Delta das Opções)', 'font':{'color':'white','size':18}, 'x':0.5}, 'shapes':[spot_line, hline0] + range_lines, 'annotations':[spot_label, range_low_label, range_high_label]}]),
    dict(label='Delta Acumulado', method='update',
         args=[{'visible':[False, True, False, False, False, False, False, False, False, False, False, False, False]},
               {'title':{'text':'EDI — Delta Acumulado (Líquido)', 'font':{'color':'white','size':18}, 'x':0.5}, 'shapes':[spot_line, hline0] + range_lines, 'annotations':[spot_label, range_low_label, range_high_label]}]),
    dict(label='GEX', method='update',
         args=[{'visible':[False, False, True, True, False, False, False, False, False, False, False, False, False]},
               {'title':{'text':'EDI — Painel Delta & GEX', 'font':{'color':'white','size':18}, 'x':0.5},
                'shapes':[spot_line, hline0] + range_lines + ([flip_line] if flip_line else []),
                'annotations':[spot_label, range_low_label, range_high_label] + ([flip_label] if flip_label else [])}]),
    dict(label='GEX (OI)', method='update',
         args=[{'visible':[False, False, False, False, False, False, False, False, False, False, True, False, False, True, False]},
               {'title':{'text':'EDI — Painel Delta & GEX', 'font':{'color':'white','size':18}, 'x':0.5},
                'shapes':[spot_line, hline0] + range_lines + ([flip_line] if flip_line else []),
                'annotations':[spot_label, range_low_label, range_high_label] + ([flip_label] if flip_label else [])}]),
    dict(label='GEX (IV)', method='update',
         args=[{'visible':[False, False, False, False, False, False, False, False, False, False, False, True, False, False, True]},
               {'title':{'text':'EDI — Painel Delta & GEX', 'font':{'color':'white','size':18}, 'x':0.5},
                'shapes':[spot_line, hline0] + range_lines + ([flip_line] if flip_line else []),
                'annotations':[spot_label, range_low_label, range_high_label] + ([flip_label] if flip_label else [])}]),
    dict(label='R Gamma (PVOP)', method='update',
         args=[{'visible':[False, False, False, False, False, False, False, False, False, False, False, False, True]},
               {'title':{'text':'EDI — R Gamma (PVOP assinado)', 'font':{'color':'white','size':18}, 'x':0.5},
                'shapes':[spot_line, hline0] + range_lines + ([flip_line] if flip_line else []),
                'annotations':[infobox3(), spot_label, range_low_label, range_high_label] + ([flip_label] if flip_label else [])}]),
    dict(label='Open Interest por Strike', method='update',
         args=[{'visible':[False, False, False, False, True, True, True, True, False, False, False, False, False]},
               {'title':{'text':'EDI — Open Interest (OI) por Strike', 'font':{'color':'white','size':18}, 'x':0.5},
                'shapes':[spot_line, hline0] + range_lines + wall_lines + ([flip_line] if flip_line else []),
                'annotations':[spot_label, range_low_label, range_high_label] + ([flip_label] if flip_label else [])}]),
    dict(label='Charm Exposure', method='update',
         args=[{'visible':[False, False, False, False, False, False, False, False, True, False, False, False, False]},
               {'title':{'text':'EDI — Charm Exposure', 'font':{'color':'white','size':18}, 'x':0.5},
                'shapes':[spot_line, hline0] + range_lines,
                'annotations':[spot_label, range_low_label, range_high_label]}]),
    dict(label='Vanna Exposure', method='update',
         args=[{'visible':[False, False, False, False, False, False, False, False, False, True, False, False, False]},
               {'title':{'text':'EDI — Vanna Exposure', 'font':{'color':'white','size':18}, 'x':0.5},
                'shapes':[spot_line, hline0] + range_lines,
                'annotations':[spot_label, range_low_label, range_high_label]}]),
    dict(label='Visão Completa', method='relayout',
         args=[{'xaxis.range':[float(min_k) - pad, float(max_k) + pad]}]),
]
overlayButtonsNew = [
    dict(label='Strikes + Midwalls + Fibonacci', method='relayout',
         args=[{'title':{'text':'EDI — Painel Delta & GEX', 'font':{'color':'white','size':18}, 'x':0.5}, 'shapes': shapes_strike_mid_fibo, 'annotations': annos_strike_mid_fibo}]),
    dict(label='Range + Walls', method='relayout',
         args=[{'title':{'text':'EDI — Painel Delta & GEX', 'font':{'color':'white','size':18}, 'x':0.5}, 'shapes':[spot_line, hline0] + range_lines + wall_lines}]),
    dict(label='Range apenas', method='relayout',
         args=[{'title':{'text':'EDI — Painel Delta & GEX', 'font':{'color':'white','size':18}, 'x':0.5}, 'shapes':[spot_line, hline0] + range_lines}]),
    dict(label='Walls apenas', method='relayout',
         args=[{'title':{'text':'EDI — Painel Delta & GEX', 'font':{'color':'white','size':18}, 'x':0.5}, 'shapes':[spot_line, hline0] + wall_lines}]),
    # Removidos botões de Gamma Flip da Figura 3 — dedicados à Figura 4
    dict(label='Limpar overlays', method='relayout',
         args=[{'title':{'text':'EDI — Painel Delta & GEX', 'font':{'color':'white','size':18}, 'x':0.5}, 'shapes':[spot_line, hline0]}]),
]
buttonsUnified = buttonsMainNew + overlayButtonsNew

BRAND_TITLE = 'EDI — Painel Delta & GEX'
fig3.update_layout(
    updatemenus=[
        dict(type='dropdown', direction='down', showactive=True, active=0, x=1.00, y=1.25, xanchor='right', yanchor='top',
             buttons=buttonsUnified, bgcolor='rgba(30,30,30,0.95)', bordercolor='#444',
             borderwidth=1, font=dict(color='#e5e7eb', size=12), pad=dict(t=4, r=4, b=4, l=4)),
        dict(type='buttons', direction='right', showactive=True, x=0.1, y=1.25, xanchor='right', yanchor='top',
             buttons=[
                 dict(label='Info ON', method='relayout', args=[{'title':{'text':'EDI — Painel Delta & GEX', 'font':{'color':'white','size':18}, 'x':0.5}, 'annotations':[infobox3(), spot_label, range_low_label, range_high_label]}]),
                 dict(label='Info OFF', method='relayout', args=[{'title':{'text':'EDI — Painel Delta & GEX', 'font':{'color':'white','size':18}, 'x':0.5}, 'annotations':[spot_label, range_low_label, range_high_label]}])
             ], bgcolor='rgba(30,30,30,0.85)', bordercolor='#444', borderwidth=1,
             font=dict(color='#e5e7eb', size=11), pad=dict(t=2, r=2, b=2, l=2))
        ,
        dict(type='buttons', direction='right', showactive=True, x=0.3, y=1.25, xanchor='right', yanchor='top',
             buttons=[
                 dict(label='Modo Básico', method='relayout', args=[{'title':{'text':'EDI — Painel Delta & GEX', 'font':{'color':'white','size':18}, 'x':0.5}, 'shapes':[spot_line, hline0], 'annotations':[spot_label]}]),
                 dict(label='Modo Avançado', method='relayout', args=[{'title':{'text':'EDI — Painel Delta & GEX', 'font':{'color':'white','size':18}, 'x':0.5}, 'shapes':[spot_line, hline0] + range_lines, 'annotations':[spot_label, range_low_label, range_high_label]}])
             ], bgcolor='rgba(30,30,30,0.85)', bordercolor='#444', borderwidth=1,
             font=dict(color='#e5e7eb', size=11), pad=dict(t=2, r=2, b=2, l=2))
    ],
    sliders=[dict(active=1, currentvalue=dict(prefix='Strikes: ~'),
                  steps=[
                      dict(label='40', method='relayout', args=[{'title':{'text':'EDI — Painel Delta & GEX', 'font':{'color':'white','size':18}, 'x':0.5}, 'xaxis.range':[float(SPOT) - (np.median(np.diff(strikes_ref)) if len(strikes_ref)>1 else 25.0)*20, float(SPOT) + (np.median(np.diff(strikes_ref)) if len(strikes_ref)>1 else 25.0)*20]}]),
                      dict(label='60', method='relayout', args=[{'title':{'text':'EDI — Painel Delta & GEX', 'font':{'color':'white','size':18}, 'x':0.5}, 'xaxis.range':[float(SPOT) - (np.median(np.diff(strikes_ref)) if len(strikes_ref)>1 else 25.0)*30, float(SPOT) + (np.median(np.diff(strikes_ref)) if len(strikes_ref)>1 else 25.0)*30]}]),
                      dict(label='80', method='relayout', args=[{'title':{'text':'EDI — Painel Delta & GEX', 'font':{'color':'white','size':18}, 'x':0.5}, 'xaxis.range':[float(SPOT) - (np.median(np.diff(strikes_ref)) if len(strikes_ref)>1 else 25.0)*40, float(SPOT) + (np.median(np.diff(strikes_ref)) if len(strikes_ref)>1 else 25.0)*40]}])
                  ]
                 )],
    title=dict(text=BRAND_TITLE, font=dict(color='white', size=18), x=0.5),
    height=int(650),
    margin=dict(t=110)
)
fig3.show()

# Métricas adicionais inspiradas no painel
net_gex_oi = float(np.nansum(gex_oi))
net_gex_vol = float(np.nansum(gex_vol))
gex_oi_signed = gC*oi_call_ref*sgn_call + gP*oi_put_ref*sgn_put
gex_vol_signed = gex_oi_signed * ivw
major_pos_oi = float(strikes_ref[int(np.argmax(gex_oi_signed))]) if len(strikes_ref)>0 else None
major_neg_oi = float(strikes_ref[int(np.argmin(gex_oi_signed))]) if len(strikes_ref)>0 else None
major_pos_vol = float(strikes_ref[int(np.argmax(gex_vol_signed))]) if len(strikes_ref)>0 else None
major_neg_vol = float(strikes_ref[int(np.argmin(gex_vol_signed))]) if len(strikes_ref)>0 else None
long_gamma = float(np.nanmean(gex_flip_base[strikes_ref>=SPOT])) if np.any(strikes_ref>=SPOT) else None
short_gamma = float(np.nanmean(gex_flip_base[strikes_ref<=SPOT])) if np.any(strikes_ref<=SPOT) else None
items = ['Spot','Delta Agregado','Volatilidade Diária (%)','Linha amarela (range)','Range baixo','Range alto','Gamma Flip','Regime','Put/Call','CALL walls top','PUT walls top','CALL wall próxima','PUT wall próxima','Midwalls acima (3)','Midwalls abaixo (3)','Net GEX (OI)','Net GEX (VOL)','Major + (GEX OI)','Major − (GEX OI)','Major + (GEX VOL)','Major − (GEX VOL)','Long Gamma (>= spot)','Short Gamma (<= spot)']
values = [f'{SPOT:.0f}', f'{delta_agregado_val:,.0f}', f'{IV_DAILY*100:.2f}', f'{range_low:.0f}–{range_high:.0f}', f'{range_low:.0f}', f'{range_high:.0f}',
          (f'{gamma_flip:.0f}' if gamma_flip is not None else 'N/A'),
          regime, (f'{pcr:.2f}' if not np.isnan(pcr) else 'N/A'),
          walls_call_txt, walls_put_txt, nearest_call_txt, nearest_put_txt, mid_above_txt, mid_below_txt,
          f'{net_gex_oi:.4f}', f'{net_gex_vol:.4f}',
          (f'{major_pos_oi:.0f}' if not np.isnan(major_pos_oi) else 'N/A'),
          (f'{major_neg_oi:.0f}' if not np.isnan(major_neg_oi) else 'N/A'),
          (f'{major_pos_vol:.0f}' if not np.isnan(major_pos_vol) else 'N/A'),
          (f'{major_neg_vol:.0f}' if not np.isnan(major_neg_vol) else 'N/A'),
          (f'{long_gamma:.4f}' if not np.isnan(long_gamma) else 'N/A'),
          (f'{short_gamma:.4f}' if not np.isnan(short_gamma) else 'N/A')]
descs  = ['Preço à vista (pontos)', 'Soma líquida de Delta por strike (Δ*OI)', 'IV diária em % (ATM por strike)', 'Intervalo diário esperado (amarelo)', 'Limite inferior esperado intradiário', 'Limite superior esperado intradiário',
          'Zero Gamma (Gamma Flip) interpolado', 'Sinal do Gamma acumulado no spot', 'Put/Call Ratio agregado',
          'Top-3 paredes de OI em CALL', 'Top-3 paredes de OI em PUT', 'Strike de CALL mais próximo do spot',
          'Strike de PUT mais próximo do spot', '3 midpoints de strike acima do spot', '3 midpoints de strike abaixo do spot',
          'Soma líquida do GEX OI', 'Soma líquida do GEX VOL', 'Strike com maior GEX OI (assinado)', 'Strike com menor GEX OI (assinado)',
          'Strike com maior GEX VOL (assinado)', 'Strike com menor GEX VOL (assinado)',
          'Média de gamma assinado acima do spot', 'Média de gamma assinado abaixo do spot']
fig_vals3 = go.Figure(data=[go.Table(
    header=dict(values=['Item','Valor','Descrição'], fill_color='grey', align='left', height=32, font=dict(color='white', size=13)),
    cells=dict(values=[items, values, descs], fill_color='black', align='left', height=26, font=dict(color='white', size=12))
)])
fig_vals3.update_layout(template='plotly_dark', margin=dict(t=30,b=30), height=700)
fig_vals3.show()

In [11]:
# Gerando o código para o Profit da Nelogica NTSL.
import numpy as np

TAMANHO_FONTE = 8
MOSTRAR_PLUS  = True
MOSTRAR_PLUS2 = True

strikes = np.unique(np.sort(np.array(strikes_ref, dtype=float)))
oi_call = np.array(oi_call_ref, dtype=float)
oi_put  = np.array(oi_put_ref, dtype=float)

call_walls_all = np.unique(np.sort(np.array(strikes_ref, dtype=float)[oi_call > 0]))
put_walls_all  = np.unique(np.sort(np.array(strikes_ref, dtype=float)[oi_put  > 0]))

midwalls_all = ((strikes[:-1] + strikes[1:]) / 2).astype(float)

IV_DAILY = float(IV_ANNUAL) / np.sqrt(252)
range_low  = float(SPOT * (1 - IV_DAILY))
range_high = float(SPOT * (1 + IV_DAILY))

def compute_gamma_flip(strikes_arr, gex_cum_arr, spot):
    if len(strikes_arr) == 0 or len(gex_cum_arr) == 0:
        return float(spot)
    sg = np.sign(gex_cum_arr)
    idx = np.where(np.diff(sg)!=0)[0]
    if len(idx)>0:
        zgs=[]
        for i in idx:
            y1, y2 = gex_cum_arr[i], gex_cum_arr[i+1]
            x1, x2 = strikes_arr[i], strikes_arr[i+1]
            zgs.append(float(x1 if y2==y1 else x1 - y1*(x2 - x1)/(y2 - y1)))
        zgs = np.array(zgs, dtype=float)
        return float(zgs[np.argmin(np.abs(zgs - spot))])
    i = int(np.argmin(np.abs(gex_cum_arr)))
    return float(strikes_arr[i])

def zero_cross_spline(strikes_arr, y_arr, spot):
    ks = np.array(strikes_arr,dtype=float)
    ys = np.array(y_arr,dtype=float)
    order = np.argsort(ks); ks=ks[order]; ys=ys[order]
    if len(ks)<3:
        return compute_gamma_flip(ks, ys, spot)
    f = PchipInterpolator(ks, ys)
    sg = np.sign(ys)
    idx = np.where(np.diff(sg)!=0)[0]
    if len(idx)==0:
        i=int(np.argmin(np.abs(ys))); return float(ks[i])
    j = int(np.argmin(np.abs(ks[idx] - float(spot))))
    i = idx[j]; a,b = ks[i], ks[i+1]
    try:
        root = brentq(lambda x: float(f(x)), float(a), float(b))
        return float(root)
    except Exception:
        return compute_gamma_flip(ks, ys, spot)

def compute_gamma_flip_sigma_kernel(strikes_arr,gex_arr,spot,iv_by_strike,T,sigma_factor):
    ks=np.array(strikes_arr,dtype=float)
    gex=np.array(gex_arr,dtype=float)
    iv=np.array(iv_by_strike,dtype=float)
    iv=np.where(iv<=0, np.nanmedian(iv[iv>0]), iv)
    z=np.log(ks/float(spot))/(iv*np.sqrt(T))
    w=np.exp(-(z**2)/(2.0*(float(sigma_factor)**2)))
    gex_cum=np.cumsum(gex*w)
    return zero_cross_spline(ks, gex_cum, spot)

def compute_gamma_flip_topn(strikes_arr,gex_arr,oi_c,oi_p,top_n,spot):
    ks=np.array(strikes_arr,dtype=float); gex=np.array(gex_arr,dtype=float)
    oi=np.array(oi_c,dtype=float)+np.array(oi_p,dtype=float)
    order=np.argsort(ks); ks=ks[order]; gex=gex[order]; oi=oi[order]
    idx=np.argsort(-oi)[:int(top_n)]
    mask=np.zeros_like(oi,dtype=bool); mask[idx]=True
    gex_sel=np.where(mask, gex, 0.0)
    gex_cum=np.cumsum(gex_sel)
    return compute_gamma_flip(ks, gex_cum, spot)

def compute_gamma_flip_hvl(strikes_arr, gex_arr, spot, hvl_daily, sigma_factor):
    if len(strikes_arr) == 0 or len(gex_arr) == 0:
        return None
    order = np.argsort(np.array(strikes_arr, dtype=float))
    ks = np.array(strikes_arr, dtype=float)[order]
    gex = np.array(gex_arr, dtype=float)[order]
    step = float(np.median(np.diff(ks))) if len(ks)>1 else 25.0
    sigma_pts = float(sigma_factor) * max(step * 2.0, float(spot) * float(hvl_daily))
    w = np.exp(-((ks - float(spot))**2) / (2.0 * (sigma_pts**2)))
    gex_cum_hvl = np.cumsum(gex * w)
    sg = np.sign(gex_cum_hvl)
    idx = np.where(np.diff(sg)!=0)[0]
    if len(idx)>0:
        j = int(np.argmin(np.abs(ks[idx] - float(spot))))
        i = idx[j]
        y1, y2 = gex_cum_hvl[i], gex_cum_hvl[i+1]
        x1, x2 = ks[i], ks[i+1]
        return float(x1 if y2==y1 else x1 - y1*(x2 - x1)/(y2 - y1))
    k_idx = int(np.argmin(np.abs(gex_cum_hvl)))
    return float(ks[k_idx])

def compute_gamma_flip_hvl_window(strikes_arr, gex_arr, spot, hvl_daily, sigma_factor):
    if len(strikes_arr) == 0 or len(gex_arr) == 0:
        return None
    order = np.argsort(np.array(strikes_arr, dtype=float))
    ks_all = np.array(strikes_arr, dtype=float)[order]
    gex_all = np.array(gex_arr, dtype=float)[order]
    step = float(np.median(np.diff(ks_all))) if len(ks_all)>1 else 25.0
    W = float(sigma_factor) * max(step * 2.0, float(spot) * float(hvl_daily))
    mask = (ks_all >= float(spot) - W) & (ks_all <= float(spot) + W)
    ks = ks_all[mask]
    gex = gex_all[mask]
    if len(ks) < 2:
        return None
    gex_cum = np.cumsum(gex)
    sg = np.sign(gex_cum)
    idx = np.where(np.diff(sg)!=0)[0]
    if len(idx)>0:
        j = int(np.argmin(np.abs(ks[idx] - float(spot))))
        i = idx[j]
        y1, y2 = gex_cum[i], gex_cum[i+1]
        x1, x2 = ks[i], ks[i+1]
        return float(x1 if y2==y1 else x1 - y1*(x2 - x1)/(y2 - y1))
    k_idx = int(np.argmin(np.abs(gex_cum)))
    return float(ks[k_idx])

def compute_gamma_flip_hvl_log(strikes_arr, gex_arr, spot, hvl_daily, sigma_factor):
    if len(strikes_arr) == 0 or len(gex_arr) == 0:
        return None
    order = np.argsort(np.array(strikes_arr, dtype=float))
    ks = np.array(strikes_arr, dtype=float)[order]
    gex = np.array(gex_arr, dtype=float)[order]
    sigma_m = float(hvl_daily) * float(sigma_factor)
    z = np.log(ks/float(spot))
    w = np.exp(-(z**2) / (2.0 * (sigma_m**2)))
    gex_cum_log = np.cumsum(gex * w)
    sg = np.sign(gex_cum_log)
    idx = np.where(np.diff(sg)!=0)[0]
    if len(idx)>0:
        j = int(np.argmin(np.abs(ks[idx] - float(spot))))
        i = idx[j]
        y1, y2 = gex_cum_log[i], gex_cum_log[i+1]
        x1, x2 = ks[i], ks[i+1]
        return float(x1 if y2==y1 else x1 - y1*(x2 - x1)/(y2 - y1))
    k_idx = int(np.argmin(np.abs(gex_cum_log)))
    return float(ks[k_idx])

gamma_flip = compute_gamma_flip(np.array(strikes_ref,dtype=float),
                                np.array(gex_cum_signed,dtype=float),
                                float(SPOT))
HVL_DAILY = float(HVL_ANNUAL)/np.sqrt(252)
gamma_flip_hvl = compute_gamma_flip_hvl(np.array(strikes_ref,dtype=float),
                                         np.array(gex_flip_base,dtype=float),
                                         float(SPOT), float(HVL_DAILY), float(SIGMA_FACTOR))
gamma_flip_hvl_log = compute_gamma_flip_hvl_log(np.array(strikes_ref,dtype=float),
                                              np.array(gex_flip_base,dtype=float),
                                              float(SPOT), float(HVL_DAILY), float(SIGMA_FACTOR))
gamma_flip_hvl_win = compute_gamma_flip_hvl_window(np.array(strikes_ref,dtype=float),
                                                   np.array(gex_flip_base,dtype=float),
                                                   float(SPOT), float(HVL_DAILY), float(SIGMA_FACTOR))
# (Figura 3) sem sweep — varredura deslocada para Figura 4 dedicada

def as_price(v): return int(round(float(v)))

lines = []
lines.append("Input")
lines.append(f"  TamanhoFonte({TAMANHO_FONTE}); // Tamanho da fonte para as linhas")
lines.append(f"  MostrarPLUS({str(MOSTRAR_PLUS).lower()});  //função para mostrar os niveis de fibo  0.382 e 0.618")
lines.append(f"  MostrarPLUS2({str(MOSTRAR_PLUS2).lower()});  //função para mostrar os niveis de fibo  0.236 e 0.764")
lines.append("")
lines.append("Inicio")

if MOSTRAR_PLUS:
    lines.append("  if MostrarPLUS then")
    lines.append("    begin")
    for i in range(len(strikes)-1):
        lower = float(strikes[i]); upper = float(strikes[i+1]); dist = upper - lower
        for p in [0.382, 0.618]:
            lvl = lower + p*dist
            lines.append(f"      HorizontalLineCustom({as_price(lvl)}, clgray, 1, psDash, \"Edi_Wall\", TamanhoFonte, tpTopLeft, CurrentDate, 0); //valores das fibonacci 0.382 e 0.618")
    lines.append("    end;")

if MOSTRAR_PLUS2:
    lines.append("  if MostrarPLUS2 then")
    lines.append("    begin")
    for i in range(len(strikes)-1):
        lower = float(strikes[i]); upper = float(strikes[i+1]); dist = upper - lower
        for p in [0.236, 0.764]:
            lvl = lower + p*dist
            lines.append(f"      HorizontalLineCustom({as_price(lvl)}, cldkgray, 1, psDash, \"Edi_Wall\", TamanhoFonte, tpTopLeft, CurrentDate, 0); //valores das fibonacci 0.236 e 0.764")
    lines.append("    end;")

for k in call_walls_all:
    lines.append(f"  HorizontalLineCustom({as_price(k)}, clblue, 1, psDash, \"CallWall\", TamanhoFonte, tpTopLeft, 0); // valores dos strikes para call")
for k in put_walls_all:
    lines.append(f"  HorizontalLineCustom({as_price(k)}, clred,  1, psDash, \"PutWall\", TamanhoFonte, tpTopLeft, 0); // valores dos strikes para put")

for k in midwalls_all:
    lines.append(f"  HorizontalLineCustom({as_price(k)}, clCream, 1, psDash, \"Edi_Wall\", TamanhoFonte, tpTopLeft, CurrentDate, 0); // valores dos Midwalls")

lines.append(f"  HorizontalLineCustom({as_price(range_high)}, cllime, 1, psDash, \"Edi_1D_MAX\", TamanhoFonte, tpTopLeft, 0); // máxima diária")
lines.append(f"  HorizontalLineCustom({as_price(range_low)},  clred,  1, psDash, \"Edi_1D_MIN\", TamanhoFonte, tpTopLeft, 0); // Minima Diária")

lines.append(f"  HorizontalLineCustom({as_price(gamma_flip)}, clfuchsia, 2, psDash, \"Edi_Gamma_Flip\", TamanhoFonte, tpTopRight, 0);")
# (Profit) Mantém apenas o Gamma Flip clássico

lines.append("")
lines.append("Fim;")

output_script = "\n".join(lines)
print(output_script)

Input
  TamanhoFonte(8); // Tamanho da fonte para as linhas
  MostrarPLUS(true);  //função para mostrar os niveis de fibo  0.382 e 0.618
  MostrarPLUS2(true);  //função para mostrar os niveis de fibo  0.236 e 0.764

Inicio
  if MostrarPLUS then
    begin
      HorizontalLineCustom(5254, clgray, 1, psDash, "Edi_Wall", TamanhoFonte, tpTopLeft, CurrentDate, 0); //valores das fibonacci 0.382 e 0.618
      HorizontalLineCustom(5271, clgray, 1, psDash, "Edi_Wall", TamanhoFonte, tpTopLeft, CurrentDate, 0); //valores das fibonacci 0.382 e 0.618
      HorizontalLineCustom(5310, clgray, 1, psDash, "Edi_Wall", TamanhoFonte, tpTopLeft, CurrentDate, 0); //valores das fibonacci 0.382 e 0.618
      HorizontalLineCustom(5315, clgray, 1, psDash, "Edi_Wall", TamanhoFonte, tpTopLeft, CurrentDate, 0); //valores das fibonacci 0.382 e 0.618
      HorizontalLineCustom(5335, clgray, 1, psDash, "Edi_Wall", TamanhoFonte, tpTopLeft, CurrentDate, 0); //valores das fibonacci 0.382 e 0.618
      HorizontalLineCusto

In [12]:
import numpy as np
import plotly.graph_objects as go
from scipy.stats import norm

def greeks(S, K, T, r, sigma, t):
    if S<=0 or K<=0 or T<=0 or sigma<=0: return np.nan, np.nan
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    delta = norm.cdf(d1) if t=='C' else norm.cdf(d1) - 1
    gamma = norm.pdf(d1) / (S*sigma*np.sqrt(T))
    return delta, gamma

def compute_gamma_flip(strikes_arr, gex_cum_arr, spot):
    if len(strikes_arr)==0 or len(gex_cum_arr)==0: return float(spot)
    sg = np.sign(gex_cum_arr)
    idx = np.where(np.diff(sg)!=0)[0]
    if len(idx)>0:
        zgs=[]
        for i in idx:
            y1,y2=gex_cum_arr[i],gex_cum_arr[i+1]
            x1,x2=strikes_arr[i],strikes_arr[i+1]
            zgs.append(float(x1 if y2==y1 else x1 - y1*(x2-x1)/(y2-y1)))
        zgs=np.array(zgs,dtype=float)
        return float(zgs[np.argmin(np.abs(zgs - spot))])
    i=int(np.argmin(np.abs(gex_cum_arr)))
    return float(strikes_arr[i])

def compute_gamma_flip_hvl(strikes_arr,gex_arr,spot,hvl_daily,sigma_factor):
    if len(strikes_arr)==0 or len(gex_arr)==0: return None
    order=np.argsort(np.array(strikes_arr,dtype=float))
    ks=np.array(strikes_arr,dtype=float)[order]
    gex=np.array(gex_arr,dtype=float)[order]
    step=float(np.median(np.diff(ks))) if len(ks)>1 else 25.0
    sigma_pts=float(sigma_factor)*max(step*2.0,float(spot)*float(hvl_daily))
    w=np.exp(-((ks-float(spot))**2)/(2.0*(sigma_pts**2)))
    gex_cum_hvl=np.cumsum(gex*w)
    sg=np.sign(gex_cum_hvl)
    idx=np.where(np.diff(sg)!=0)[0]
    if len(idx)>0:
        j=int(np.argmin(np.abs(ks[idx]-float(spot))))
        i=idx[j]
        y1,y2=gex_cum_hvl[i],gex_cum_hvl[i+1]
        x1,x2=ks[i],ks[i+1]
        return float(x1 if y2==y1 else x1 - y1*(x2-x1)/(y2-y1))
    k_idx=int(np.argmin(np.abs(gex_cum_hvl)))
    return float(ks[k_idx])

def compute_gamma_flip_hvl_log(strikes_arr,gex_arr,spot,hvl_daily,sigma_factor):
    if len(strikes_arr)==0 or len(gex_arr)==0: return None
    order=np.argsort(np.array(strikes_arr,dtype=float))
    ks=np.array(strikes_arr,dtype=float)[order]
    gex=np.array(gex_arr,dtype=float)[order]
    sigma_m=float(hvl_daily)*float(sigma_factor)
    z=np.log(ks/float(spot))
    w=np.exp(-(z**2)/(2.0*(sigma_m**2)))
    gex_cum_log=np.cumsum(gex*w)
    sg=np.sign(gex_cum_log)
    idx=np.where(np.diff(sg)!=0)[0]
    if len(idx)>0:
        j=int(np.argmin(np.abs(ks[idx]-float(spot))))
        i=idx[j]
        y1,y2=gex_cum_log[i],gex_cum_log[i+1]
        x1,x2=ks[i],ks[i+1]
        return float(x1 if y2==y1 else x1 - y1*(x2-x1)/(y2-y1))
    k_idx=int(np.argmin(np.abs(gex_cum_log)))
    return float(ks[k_idx])

def compute_gamma_flip_hvl_window(strikes_arr,gex_arr,spot,hvl_daily,sigma_factor):
    if len(strikes_arr)==0 or len(gex_arr)==0: return None
    order=np.argsort(np.array(strikes_arr,dtype=float))
    ks_all=np.array(strikes_arr,dtype=float)[order]
    gex_all=np.array(gex_arr,dtype=float)[order]
    step=float(np.median(np.diff(ks_all))) if len(ks_all)>1 else 25.0
    W=float(sigma_factor)*max(step*2.0,float(spot)*float(hvl_daily))
    mask=(ks_all>=float(spot)-W) & (ks_all<=float(spot)+W)
    ks=ks_all[mask]; gex=gex_all[mask]
    if len(ks)<2: return None
    gex_cum=np.cumsum(gex)
    sg=np.sign(gex_cum)
    idx=np.where(np.diff(sg)!=0)[0]
    if len(idx)>0:
        j=int(np.argmin(np.abs(ks[idx]-float(spot))))
        i=idx[j]
        y1,y2=gex_cum[i],gex_cum[i+1]
        x1,x2=ks[i],ks[i+1]
        return float(x1 if y2==y1 else x1 - y1*(x2-x1)/(y2-y1))
    k_idx=int(np.argmin(np.abs(gex_cum)))
    return float(ks[k_idx])

gC,gP=[],[]
for K in strikes_ref:
    _,g=greeks(SPOT,K,T,RISK_FREE,IV_ANNUAL,'C'); gC.append(0 if np.isnan(g) else g)
    _,g=greeks(SPOT,K,T,RISK_FREE,IV_ANNUAL,'P'); gP.append(0 if np.isnan(g) else g)
gC,gP=np.array(gC),np.array(gP)

CONTRACT_MULT=float(CONTRACT_MULT)
gex_tot=gC*oi_call_ref*CONTRACT_MULT*SPOT*0.01 + gP*oi_put_ref*CONTRACT_MULT*SPOT*0.01
gex_cum=np.cumsum(gex_tot)

sgn_call=np.where(strikes_ref<=SPOT,+1.0,-1.0)
sgn_put=np.where(strikes_ref>=SPOT,-1.0,+1.0)
gex_flip_base=(gC*oi_call_ref*sgn_call + gP*oi_put_ref*sgn_put)*CONTRACT_MULT*SPOT*0.01
gex_cum_signed=np.cumsum(gex_flip_base)

ivw=iv_strike_ref if 'iv_strike_ref' in globals() else np.ones_like(strikes_ref,dtype=float)
gex_flip_base_iv=gex_flip_base*ivw
gex_cum_signed_iv=np.cumsum(gex_flip_base_iv)

PVOP=float(PVOP) if 'PVOP' in globals() else float(CONTRACT_MULT)*float(SPOT)*0.01
gex_flip_base_pvop=(gC*oi_call_ref*sgn_call + gP*oi_put_ref*sgn_put)*PVOP
gex_cum_signed_pvop=np.cumsum(gex_flip_base_pvop)

HVL_DAILY=float(HVL_ANNUAL)/np.sqrt(252)
min_k,max_k=float(np.min(strikes_ref)),float(np.max(strikes_ref))

fig4=go.Figure()
fig4.add_trace(go.Bar(x=strikes_ref,y=gex_flip_base,name='Gamma Exposure (assinado)',marker_color='cyan',opacity=0.6,visible=True,hovertemplate='Strike %{x:.0f}<br>Valor %{y:.0f}'))
fig4.add_trace(go.Scatter(x=strikes_ref,y=gex_cum_signed,mode='lines',name='Acumulado (assinado)',line=dict(color='orange',width=3),visible=True,hovertemplate='Strike %{x:.0f}<br>Valor %{y:.0f}'))
fig4.add_trace(go.Bar(x=strikes_ref,y=gex_tot,name='Gamma Exposure (tot)',marker_color='lightblue',opacity=0.4,visible=False,hovertemplate='Strike %{x:.0f}<br>Valor %{y:.0f}'))
fig4.add_trace(go.Scatter(x=strikes_ref,y=gex_cum,mode='lines',name='Acumulado (tot)',line=dict(color='yellow',width=2),visible=False,hovertemplate='Strike %{x:.0f}<br>Valor %{y:.0f}'))
fig4.add_trace(go.Bar(x=strikes_ref,y=gex_flip_base_iv,name='Gamma Exposure (assinado IV)',marker_color='purple',opacity=0.6,visible=False,hovertemplate='Strike %{x:.0f}<br>Valor %{y:.0f}'))
fig4.add_trace(go.Scatter(x=strikes_ref,y=gex_cum_signed_iv,mode='lines',name='Acumulado (assinado IV)',line=dict(color='violet',width=3),visible=False,hovertemplate='Strike %{x:.0f}<br>Valor %{y:.0f}'))
fig4.add_trace(go.Bar(x=strikes_ref,y=gex_flip_base_pvop,name='Gamma Exposure (assinado PVOP)',marker_color='teal',opacity=0.6,visible=False,hovertemplate='Strike %{x:.0f}<br>Valor %{y:.0f}'))
fig4.add_trace(go.Scatter(x=strikes_ref,y=gex_cum_signed_pvop,mode='lines',name='Acumulado (assinado PVOP)',line=dict(color='lightgreen',width=3),visible=False,hovertemplate='Strike %{x:.0f}<br>Valor %{y:.0f}'))

fig4.update_layout(template='plotly_dark',barmode='overlay',
                   xaxis_title='Strike',yaxis_title='Exposição (normalizada)',
                   xaxis=dict(range=[SPOT-300,SPOT+300]),
                   legend=dict(orientation='h',yanchor='bottom',y=1.02,xanchor='left',x=0.0),
                   margin=dict(t=100))
spot_line4=dict(type='line',x0=float(SPOT),x1=float(SPOT),y0=0,y1=1,xref='x',yref='paper',line=dict(color='lime',dash='dot',width=2))
hline04=dict(type='line',x0=float(min_k),x1=float(max_k),y0=0,y1=0,line=dict(color='white',dash='dot',width=1))

flip_classic=compute_gamma_flip(np.array(strikes_ref,dtype=float),np.array(gex_cum_signed,dtype=float),float(SPOT))
flip_classic_spl=zero_cross_spline(np.array(strikes_ref,dtype=float),np.array(gex_cum_signed,dtype=float),float(SPOT))
flip_hvl_pts=compute_gamma_flip_hvl(np.array(strikes_ref,dtype=float),np.array(gex_flip_base,dtype=float),float(SPOT),float(HVL_DAILY),float(SIGMA_FACTOR))
flip_hvl_log=compute_gamma_flip_hvl_log(np.array(strikes_ref,dtype=float),np.array(gex_flip_base,dtype=float),float(SPOT),float(HVL_DAILY),float(SIGMA_FACTOR))
flip_hvl_win=compute_gamma_flip_hvl_window(np.array(strikes_ref,dtype=float),np.array(gex_flip_base,dtype=float),float(SPOT),float(HVL_DAILY),float(SIGMA_FACTOR))
flip_hvl_log_iv=compute_gamma_flip_hvl_log(np.array(strikes_ref,dtype=float),np.array(gex_flip_base_iv,dtype=float),float(SPOT),float(HVL_DAILY),float(SIGMA_FACTOR))
flip_sigma_kernel=compute_gamma_flip_sigma_kernel(np.array(strikes_ref,dtype=float),np.array(gex_flip_base,dtype=float),float(SPOT),np.array(ivw,dtype=float),float(T),float(SIGMA_FACTOR))
flip_topn=compute_gamma_flip_topn(np.array(strikes_ref,dtype=float),np.array(gex_flip_base,dtype=float),np.array(oi_call_ref,dtype=float),np.array(oi_put_ref,dtype=float),int(top_n) if 'top_n' in globals() else 3,float(SPOT))

flip_lines4=[]; flip_annos4=[]
def add_flip(shape_list,anno_list,x,color,text,y):
    if x is None: return
    shape_list.append(dict(type='line',x0=float(x),x1=float(x),y0=0,y1=1,xref='x',yref='paper',line=dict(color=color,dash='dash',width=2)))
    anno_list.append(dict(x=float(x),y=y,xref='x',yref='paper',text=text,showarrow=False,font=dict(color=color,size=12),bgcolor='black',bordercolor=color))
add_flip(flip_lines4,flip_annos4,flip_classic,'red','Flip (clássico)',0.06)
add_flip(flip_lines4,flip_annos4,flip_classic_spl,'tomato','Flip (clássico spline)',0.08)
add_flip(flip_lines4,flip_annos4,flip_hvl_pts,'fuchsia','Flip (HVL pontos)',0.10)
add_flip(flip_lines4,flip_annos4,flip_hvl_log,'deeppink','Flip (HVL log)',0.14)
add_flip(flip_lines4,flip_annos4,flip_hvl_log_iv,'purple','Flip (HVL log IV)',0.16)
add_flip(flip_lines4,flip_annos4,flip_sigma_kernel,'orange','Flip (Sigma kernel)',0.18)
add_flip(flip_lines4,flip_annos4,flip_topn,'steelblue','Flip (Top-N OI)',0.20)
add_flip(flip_lines4,flip_annos4,flip_hvl_win,'magenta','Flip (HVL janela)',0.18)

SIGMA_PRESETS=[0.50,0.75,1.00]
preset_shapes={}; preset_annos={}
for sf in SIGMA_PRESETS:
    fhp=compute_gamma_flip_hvl(np.array(strikes_ref,dtype=float),np.array(gex_flip_base,dtype=float),float(SPOT),float(HVL_DAILY),float(sf))
    fhl=compute_gamma_flip_hvl_log(np.array(strikes_ref,dtype=float),np.array(gex_flip_base,dtype=float),float(SPOT),float(HVL_DAILY),float(sf))
    fhw=compute_gamma_flip_hvl_window(np.array(strikes_ref,dtype=float),np.array(gex_flip_base,dtype=float),float(SPOT),float(HVL_DAILY),float(sf))
    fhl_iv=compute_gamma_flip_hvl_log(np.array(strikes_ref,dtype=float),np.array(gex_flip_base_iv,dtype=float),float(SPOT),float(HVL_DAILY),float(sf))
    fsk=compute_gamma_flip_sigma_kernel(np.array(strikes_ref,dtype=float),np.array(gex_flip_base,dtype=float),float(SPOT),np.array(ivw,dtype=float),float(T),float(sf))
    shps=[]; ann=[]
    add_flip(shps,ann,flip_classic,'red','Flip (clássico)',0.06)
    add_flip(shps,ann,fhp,'fuchsia',f'Flip (HVL pontos sf={sf:.2f})',0.10)
    add_flip(shps,ann,fhl,'deeppink',f'Flip (HVL log sf={sf:.2f})',0.14)
    add_flip(shps,ann,fhl_iv,'purple',f'Flip (HVL log IV sf={sf:.2f})', 0.16)
    add_flip(shps,ann,fsk,'orange',f'Flip (Sigma kernel sf={sf:.2f})', 0.18)
    add_flip(shps,ann,fhw,'magenta',f'Flip (HVL janela sf={sf:.2f})', 0.20)
    preset_shapes[sf]=shps; preset_annos[sf]=ann

flip_classic_pv=compute_gamma_flip(np.array(strikes_ref,dtype=float),np.array(gex_cum_signed_pvop,dtype=float),float(SPOT))
flip_hvl_pts_pv=compute_gamma_flip_hvl(np.array(strikes_ref,dtype=float),np.array(gex_flip_base_pvop,dtype=float),float(SPOT),float(HVL_DAILY),float(SIGMA_FACTOR))
flip_hvl_log_pv=compute_gamma_flip_hvl_log(np.array(strikes_ref,dtype=float),np.array(gex_flip_base_pvop,dtype=float),float(SPOT),float(HVL_DAILY),float(SIGMA_FACTOR))
flip_hvl_win_pv=compute_gamma_flip_hvl_window(np.array(strikes_ref,dtype=float),np.array(gex_flip_base_pvop,dtype=float),float(SPOT),float(HVL_DAILY),float(SIGMA_FACTOR))
pvop_shapes=[]; pvop_annos=[]
add_flip(pvop_shapes,pvop_annos,flip_classic_pv,'red','Flip (clássico PVOP)',0.06)
add_flip(pvop_shapes,pvop_annos,flip_hvl_pts_pv,'fuchsia','Flip (HVL pontos PVOP)',0.10)
add_flip(pvop_shapes,pvop_annos,flip_hvl_log_pv,'deeppink','Flip (HVL log PVOP)',0.14)
add_flip(pvop_shapes,pvop_annos,flip_hvl_win_pv,'magenta','Flip (HVL janela PVOP)',0.18)

buttons_fig4=[
    dict(label='Assinado + flips',method='relayout',args=[{'shapes':[spot_line4,hline04]+flip_lines4,'annotations':flip_annos4}]),
    dict(label='Assinado + flips (sf=0.50)',method='relayout',args=[{'shapes':[spot_line4,hline04]+preset_shapes[0.50],'annotations':preset_annos[0.50]}]),
    dict(label='Assinado + flips (sf=0.75)',method='relayout',args=[{'shapes':[spot_line4,hline04]+preset_shapes[0.75],'annotations':preset_annos[0.75]}]),
    dict(label='Assinado + flips (sf=1.00)',method='relayout',args=[{'shapes':[spot_line4,hline04]+preset_shapes[1.00],'annotations':preset_annos[1.00]}]),
    dict(label='Assinado + flips (PVOP)',method='update',args=[{'visible':[False,False,False,False,False,False,True,True]},{'shapes':[spot_line4,hline04]+pvop_shapes,'annotations':pvop_annos}]),
    dict(label='Assinado apenas',method='update',args=[{'visible':[True,True,False,False,False,False,False,False]},{'shapes':[spot_line4,hline04],'annotations':[]}]),
    dict(label='Assinado apenas (PVOP)',method='update',args=[{'visible':[False,False,False,False,False,False,True,True]},{'shapes':[spot_line4,hline04],'annotations':[]}]),
    dict(label='Tot + cumulativo',method='update',args=[{'visible':[False,False,True,True,False,False,False,False]},{'shapes':[spot_line4,hline04],'annotations':[]}]),
]
fig4.update_layout(updatemenus=[dict(type='dropdown',direction='down',showactive=True,active=0,x=1.00,y=1.25,xanchor='right',yanchor='top',buttons=buttons_fig4,bgcolor='rgba(30,30,30,0.95)',bordercolor='#444',borderwidth=1,font=dict(color='#e5e7eb',size=12),pad=dict(t=4,r=4,b=4,l=4))])
fig4.update_layout(title=dict(text='EDI — GEX & Gamma Flip (Modelos)',font=dict(color='white',size=18),x=0.5),height=650,margin=dict(t=110))
fig4.show()

items4=['Flip (clássico)','Flip (clássico spline)','Flip (HVL pontos)','Flip (HVL log)','Flip (HVL log IV)','Flip (HVL janela)','Flip (Sigma kernel)','Flip (Top-N OI)','Flip (clássico PVOP)','Flip (HVL pontos PVOP)','Flip (HVL log PVOP)','Flip (HVL janela PVOP)']
values4=[(f'{flip_classic:.0f}' if flip_classic is not None else 'N/A'),
         (f'{flip_classic_spl:.0f}' if flip_classic_spl is not None else 'N/A'),
         (f'{flip_hvl_pts:.0f}' if flip_hvl_pts is not None else 'N/A'),
         (f'{flip_hvl_log:.0f}' if flip_hvl_log is not None else 'N/A'),
         (f'{flip_hvl_log_iv:.0f}' if flip_hvl_log_iv is not None else 'N/A'),
         (f'{flip_hvl_win:.0f}' if flip_hvl_win is not None else 'N/A'),
         (f'{flip_sigma_kernel:.0f}' if flip_sigma_kernel is not None else 'N/A'),
         (f'{flip_topn:.0f}' if flip_topn is not None else 'N/A'),
         (f'{flip_classic_pv:.0f}' if flip_classic_pv is not None else 'N/A'),
         (f'{flip_hvl_pts_pv:.0f}' if flip_hvl_pts_pv is not None else 'N/A'),
         (f'{flip_hvl_log_pv:.0f}' if flip_hvl_log_pv is not None else 'N/A'),
         (f'{flip_hvl_win_pv:.0f}' if flip_hvl_win_pv is not None else 'N/A')]
fig_vals4=go.Figure(data=[go.Table(header=dict(values=['Modelo','Valor'],fill_color='black',font=dict(color='white',size=13)),
                                   cells=dict(values=[items4,values4],fill_color='rgba(20,20,20,0.9)',font=dict(color='white',size=12)))])
fig_vals4.update_layout(template='plotly_dark',height=360,margin=dict(t=20,b=20))
fig_vals4.show()

# Interface — Botão flutuante para copiar código do Profit
import json
from IPython.display import HTML, display
btn_html = f"""
<div id="profitCopyBar" style="position:fixed; top:12px; right:12px; background:#111827; padding:10px 12px; border:1px solid #374151; border-radius:8px; box-shadow:0 4px 12px rgba(0,0,0,0.35); z-index:9999;">
  <span style="color:#e5e7eb; font-size:14px; margin-right:8px;">Copiar código para o Profit</span>
  <button id="copyProfitBtn" style="padding:6px 10px; background:#2563eb; color:white; border:0; border-radius:6px; cursor:pointer;">Copiar</button>
</div>
<script>
  const profitScript = """ + json.dumps(output_script) + """;
  const btn = document.getElementById('copyProfitBtn');
  if (btn) {
    btn.onclick = async () => {
      try {
        await navigator.clipboard.writeText(profitScript);
        alert('Código copiado para a área de transferência.');
      } catch (e) {
        console.log(e);
        alert('Falha ao copiar automaticamente. O código será exibido abaixo para copiar manualmente.');
        const ta = document.getElementById('profitTextarea');
        if (ta) { ta.value = profitScript; ta.style.display = 'block'; ta.select(); document.execCommand('copy'); }
      }
    };
  }
</script>
<textarea id="profitTextarea" style="display:none; width:100%; height:140px; margin-top:8px; background:#0b1020; color:#e5e7eb; border:1px solid #374151; border-radius:6px;"></textarea>
"""
display(HTML(btn_html))

import os, json, re
os.makedirs('exports', exist_ok=True)
fig3.update_layout(title=dict(text='EDI — Painel Delta & GEX', font=dict(color='white', size=18), x=0.5), margin=dict(t=120))
html3 = fig3.to_html(include_plotlyjs='cdn', full_html=True)
html3 = re.sub(r'<title>.*?</title>', '<title>EDI — Painel Delta & GEX</title>', html3, flags=re.S)
if '<title>' not in html3:
    html3 = re.sub(r'<head>(.*?)</head>', r'<head><title>EDI — Painel Delta & GEX</title>\1</head>', html3, flags=re.S)
with open('exports/Figura3.html','w',encoding='utf-8') as f: f.write(html3)
html3_tbl = fig_vals3.to_html(include_plotlyjs='cdn', full_html=True)
html3_tbl = re.sub(r'<title>.*?</title>', '<title>EDI — Tabela Delta & GEX</title>', html3_tbl, flags=re.S)
if '<title>' not in html3_tbl:
    html3_tbl = re.sub(r'<head>(.*?)</head>', r'<head><title>EDI — Tabela Delta & GEX</title>\1</head>', html3_tbl, flags=re.S)
with open('exports/Figura3_Tabela.html','w',encoding='utf-8') as f: f.write(html3_tbl)
fig4.update_layout(title=dict(text='EDI — GEX & Gamma Flip (Modelos)', font=dict(color='white', size=18), x=0.5), margin=dict(t=120))
html4 = fig4.to_html(include_plotlyjs='cdn', full_html=True)
html4 = re.sub(r'<title>.*?</title>', '<title>EDI — GEX & Gamma Flip (Modelos)</title>', html4, flags=re.S)
if '<title>' not in html4:
    html4 = re.sub(r'<head>(.*?)</head>', r'<head><title>EDI — GEX & Gamma Flip (Modelos)</title>\1</head>', html4, flags=re.S)
with open('exports/Figura4.html','w',encoding='utf-8') as f: f.write(html4)
html4_tbl = fig_vals4.to_html(include_plotlyjs='cdn', full_html=True)
html4_tbl = re.sub(r'<title>.*?</title>', '<title>EDI — Tabela GEX & Gamma Flip</title>', html4_tbl, flags=re.S)
if '<title>' not in html4_tbl:
    html4_tbl = re.sub(r'<head>(.*?)</head>', r'<head><title>EDI — Tabela GEX & Gamma Flip</title>\1</head>', html4_tbl, flags=re.S)
with open('exports/Figura4_Tabela.html','w',encoding='utf-8') as f: f.write(html4_tbl)
with open('exports/Profit_NTSL.txt','w',encoding='utf-8') as f: f.write(output_script)
metrics={
  'spot': float(SPOT),
  'gamma_flip': float(gamma_flip) if gamma_flip is not None else None,
  'sigma_factor': float(SIGMA_FACTOR) if 'SIGMA_FACTOR' in globals() else None,
  'top_n': int(top_n) if 'top_n' in globals() else None,
  'flip_models': {
    'classic': float(flip_classic) if flip_classic is not None else None,
    'classic_spline': float(flip_classic_spl) if flip_classic_spl is not None else None,
    'hvl_pts': float(flip_hvl_pts) if flip_hvl_pts is not None else None,
    'hvl_log': float(flip_hvl_log) if flip_hvl_log is not None else None,
    'hvl_log_iv': float(flip_hvl_log_iv) if flip_hvl_log_iv is not None else None,
    'hvl_win': float(flip_hvl_win) if flip_hvl_win is not None else None,
    'sigma_kernel': float(flip_sigma_kernel) if 'flip_sigma_kernel' in globals() and flip_sigma_kernel is not None else None,
    'topn': float(flip_topn) if 'flip_topn' in globals() and flip_topn is not None else None,
    'classic_pv': float(flip_classic_pv) if flip_classic_pv is not None else None,
    'hvl_pts_pv': float(flip_hvl_pts_pv) if flip_hvl_pts_pv is not None else None,
    'hvl_log_pv': float(flip_hvl_log_pv) if flip_hvl_log_pv is not None else None,
    'hvl_win_pv': float(flip_hvl_win_pv) if flip_hvl_win_pv is not None else None
  }
}
with open('exports/metrics.json','w',encoding='utf-8') as f: json.dump(metrics, f, ensure_ascii=False, indent=2)


In [13]:
import os
import importlib, sys
from importlib import metadata as md
try:
    pv = md.version('plotly')
except Exception:
    pv = 'desconhecida'
print('Plotly versão:', pv)
kaleido_spec = importlib.util.find_spec('kaleido')
if kaleido_spec is None:
    print('Kaleido não encontrado no kernel atual. Use: py -3 -m pip install -U kaleido')
else:
    kaleido = importlib.import_module('kaleido')
    print('Kaleido encontrado em:', kaleido.__file__)
os.makedirs('exports', exist_ok=True)
try:
    fig3.write_image('exports/Figura3.png', format='png', scale=2, width=1400, height=650)
    fig4.write_image('exports/Figura4.png', format='png', scale=2, width=1400, height=650)
except Exception as e:
    print('Falha ao gerar imagens estáticas:', e)
index_html = '''
<!doctype html><html lang="pt-br"><head><meta charset="utf-8"><title>EDI — Análise de Opções</title>
<meta name="viewport" content="width=device-width, initial-scale=1">
<style>body{background:#0b1020;color:#e5e7eb;font-family:system-ui,Segoe UI,Roboto}a{color:#93c5fd;text-decoration:none} .card{display:flex;gap:16px;align-items:center;background:#111827;border:1px solid #374151;border-radius:10px;padding:12px;margin:10px 0}</style></head><body>
<h1>USD/BRL — Painéis Técnicos</h1>
<div class="card"><img src="Figura3.png" alt="Painel Delta & GEX" style="height:120px;border-radius:6px"><div><a href="Figura3.html">Abrir Painel Delta & GEX</a><br><a href="Figura3_Tabela.html">Abrir Tabela do Painel</a></div></div>
<div class="card"><img src="Figura4.png" alt="GEX & Gamma Flip" style="height:120px;border-radius:6px"><div><a href="Figura4.html">Abrir GEX & Gamma Flip</a><br><a href="Figura4_Tabela.html">Abrir Tabela de Modelos</a></div></div>
</body></html>
'''
with open('exports/index.html','w',encoding='utf-8') as f: f.write(index_html)
agg = options.groupby(['StrikeK','OptionType'], as_index=False)['Open Int'].sum().pivot(index='StrikeK', columns='OptionType', values='Open Int').fillna(0)
agg = agg.rename(columns={'CALL':'CALL_OI','PUT':'PUT_OI'}).reset_index().rename(columns={'StrikeK':'Strike'})
agg.to_csv('exports/oi_by_strike.csv', index=False, encoding='utf-8')
try:
    fig3.write_image('exports/Painel_Delta_GEX.svg', format='svg', width=1400, height=650)
    fig4.write_image('exports/GEX_Gamma_Flip.svg', format='svg', width=1400, height=650)
except Exception as e:
    pass

Plotly versão: 6.5.0
Kaleido encontrado em: c:\Users\ednil\AppData\Local\Programs\Python\Python313\Lib\site-packages\kaleido\__init__.py
